In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/predict-energy-behavior-of-prosumers/client.csv
/kaggle/input/predict-energy-behavior-of-prosumers/gas_prices.csv
/kaggle/input/predict-energy-behavior-of-prosumers/electricity_prices.csv
/kaggle/input/predict-energy-behavior-of-prosumers/weather_station_to_county_mapping.csv
/kaggle/input/predict-energy-behavior-of-prosumers/public_timeseries_testing_util.py
/kaggle/input/predict-energy-behavior-of-prosumers/historical_weather.csv
/kaggle/input/predict-energy-behavior-of-prosumers/county_id_to_name_map.json
/kaggle/input/predict-energy-behavior-of-prosumers/train.csv
/kaggle/input/predict-energy-behavior-of-prosumers/forecast_weather.csv
/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/sample_submission.csv
/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/client.csv
/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/gas_prices.csv
/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/electricity

In [2]:
# CORE DATA MANIPULATION
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import pearsonr, spearmanr
import math

# MACHINE LEARNING - SCIKIT-LEARN
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, StratifiedKFold, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.feature_selection import SelectKBest, chi2, f_classif, RFE, RFECV
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVC, SVR
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, roc_auc_score, roc_curve, auc
from sklearn.pipeline import Pipeline

# ADVANCED ML LIBRARIES
import xgboost as xgb
from xgboost import XGBRegressor
import lightgbm as lgb
from catboost import CatBoostClassifier, CatBoostRegressor

# VISUALIZATION
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo

# CONFIGURATION FOR PLOTS
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline


# DEEP LEARNING 
try:
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Dropout, LSTM, Conv2D, MaxPooling2D, Flatten, BatchNormalization
    from tensorflow.keras.optimizers import Adam
    from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
    print("TensorFlow disponible")
except ImportError:
    print("TensorFlow no disponible en este entorno")


# UTILITIES
import os
import sys
import warnings
import itertools
from datetime import datetime, timedelta
import time
from collections import Counter
import pickle
import joblib

# JUPYTER SPECIFIC
from IPython.display import display, HTML
from tqdm.notebook import tqdm

# SUPPRESS WARNINGS
warnings.filterwarnings('ignore')

# PANDAS CONFIGURATION
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# NUMPY CONFIGURATION
np.random.seed(42)

# PLOTLY CONFIGURATION
pyo.init_notebook_mode(connected=True)

print("✅ Todas las librerías importadas correctamente!")
print(f"📊 Pandas version: {pd.__version__}")
print(f"🔢 NumPy version: {np.__version__}")
print(f"🤖 Scikit-learn version: {__import__('sklearn').__version__}")
print(f"📈 Matplotlib version: {__import__('matplotlib').__version__}")
print(f"🎨 Seaborn version: {sns.__version__}")

# ============================================================================
# FUNCIONES ÚTILES ADICIONALES
# ============================================================================

def quick_info(df):
    """Información rápida del dataset"""
    print(f"📊 Dataset Shape: {df.shape}")
    print(f"🔢 Columnas numéricas: {df.select_dtypes(include=[np.number]).columns.tolist()}")
    print(f"📝 Columnas categóricas: {df.select_dtypes(include=['object']).columns.tolist()}")
    print(f"❌ Valores nulos por columna:")
    print(df.isnull().sum()[df.isnull().sum() > 0])

def plot_missing_values(df):
    """Visualizar valores faltantes"""
    missing = df.isnull().sum()
    missing = missing[missing > 0]
    if len(missing) > 0:
        plt.figure(figsize=(10, 6))
        missing.plot(kind='bar')
        plt.title('Valores Faltantes por Columna')
        plt.ylabel('Cantidad')
        plt.xticks(rotation=45)
        plt.show()
    else:
        print("✅ No hay valores faltantes en el dataset")


2025-09-05 10:57:41.292679: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757069861.510716     179 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757069861.572769     179 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


TensorFlow disponible


✅ Todas las librerías importadas correctamente!
📊 Pandas version: 2.2.3
🔢 NumPy version: 1.26.4
🤖 Scikit-learn version: 1.2.2
📈 Matplotlib version: 3.7.2
🎨 Seaborn version: 0.12.2


In [3]:
#competencia https://www.kaggle.com/competitions/predict-energy-behavior-of-prosumers/overview

# Files
# train.csv

# county - An ID code for the county.
# is_business - Boolean for whether or not the prosumer is a business.
# product_type - ID code with the following mapping of codes to contract types: {0: "Combined", 1: "Fixed", 2: "General service", 3: "Spot"}.
# target - The consumption or production amount for the relevant segment for the hour. The segments are defined by the county, is_business, and product_type.
# is_consumption - Boolean for whether or not this row's target is consumption or production.
# datetime - The Estonian time in EET (UTC+2) / EEST (UTC+3). It describes the start of the 1-hour period on which target is given.
# data_block_id - All rows sharing the same data_block_id will be available at the same forecast time. This is a function of what information is available when forecasts are actually made, at 11 AM each morning. For example, if the forecast weather data_block_id for predictins made on October 31st is 100 then the historic weather data_block_id for October 31st will be 101 as the historic weather data is only actually available the next day.
# row_id - A unique identifier for the row.
# prediction_unit_id - A unique identifier for the county, is_business, and product_type combination. New prediction units can appear or disappear in the test set.
# gas_prices.csv

# origin_date - The date when the day-ahead prices became available.
# forecast_date - The date when the forecast prices should be relevant.
# [lowest/highest]_price_per_mwh - The lowest/highest price of natural gas that on the day ahead market that trading day, in Euros per megawatt hour equivalent.
# data_block_id
# client.csv

# product_type
# county - An ID code for the county. See county_id_to_name_map.json for the mapping of ID codes to county names.
# eic_count - The aggregated number of consumption points (EICs - European Identifier Code).
# installed_capacity - Installed photovoltaic solar panel capacity in kilowatts.
# is_business - Boolean for whether or not the prosumer is a business.
# date
# data_block_id
# electricity_prices.csv

# origin_date
# forecast_date - Represents the start of the 1-hour period when the price is valid
# euros_per_mwh - The price of electricity on the day ahead markets in euros per megawatt hour.
# data_block_id
# forecast_weather.csv Weather forecasts that would have been available at prediction time. Sourced from the European Centre for Medium-Range Weather Forecasts.

# [latitude/longitude] - The coordinates of the weather forecast.
# origin_datetime - The timestamp of when the forecast was generated.
# hours_ahead - The number of hours between the forecast generation and the forecast weather. Each forecast covers 48 hours in total.
# temperature - The air temperature at 2 meters above ground in degrees Celsius. Estimated for the end of the 1-hour period.
# dewpoint - The dew point temperature at 2 meters above ground in degrees Celsius. Estimated for the end of the 1-hour period.
# cloudcover_[low/mid/high/total] - The percentage of the sky covered by clouds in the following altitude bands: 0-2 km, 2-6, 6+, and total. Estimated for the end of the 1-hour period.
# 10_metre_[u/v]_wind_component - The [eastward/northward] component of wind speed measured 10 meters above surface in meters per second. Estimated for the end of the 1-hour period.
# data_block_id
# forecast_datetime - The timestamp of the predicted weather. Generated from origin_datetime plus hours_ahead. This represents the start of the 1-hour period for which weather data are forecasted.
# direct_solar_radiation - The direct solar radiation reaching the surface on a plane perpendicular to the direction of the Sun accumulated during the hour, in watt-hours per square meter.
# surface_solar_radiation_downwards - The solar radiation, both direct and diffuse, that reaches a horizontal plane at the surface of the Earth, accumulated during the hour, in watt-hours per square meter.
# snowfall - Snowfall over hour in units of meters of water equivalent.
# total_precipitation - The accumulated liquid, comprising rain and snow that falls on Earth's surface over the described hour, in units of meters.
# historical_weather.csv Historic weather data.

# datetime - This represents the start of the 1-hour period for which weather data are measured.
# temperature - Measured at the end of the 1-hour period.
# dewpoint - Measured at the end of the 1-hour period.
# rain - Different from the forecast conventions. The rain from large scale weather systems of the hour in millimeters.
# snowfall - Different from the forecast conventions. Snowfall over the hour in centimeters.
# surface_pressure - The air pressure at surface in hectopascals.
# cloudcover_[low/mid/high/total] - Different from the forecast conventions. Cloud cover at 0-3 km, 3-8, 8+, and total.
# windspeed_10m - Different from the forecast conventions. The wind speed at 10 meters above ground in meters per second.
# winddirection_10m - Different from the forecast conventions. The wind direction at 10 meters above ground in degrees.
# shortwave_radiation - Different from the forecast conventions. The global horizontal irradiation in watt-hours per square meter.
# direct_solar_radiation
# diffuse_radiation - Different from the forecast conventions. The diffuse solar irradiation in watt-hours per square meter.
# [latitude/longitude] - The coordinates of the weather station.
# data_block_id

In [4]:
#modelos a correr: NN por ser forecasting y algun xgb o lgb
#son 4 dias a predecir el comportamiento de los prosumers
#predecir con rolling de a 1 dia


#analizar y preparar
# como siempre primero ver los faltantes y los NaN y en cada caso decidir que hacer
# atento con valores expresados para el dia actual o la hora actual / la hora anterior / el dia de mañana , ver de no malinterpretar
# ver la correlacion de precios sea gas y/o electricidad para ver como se comparta con business y no business
# lo mismo con el clima para business y no business
# ver si aplica el supuesto de que los business si operan bajo esos valores (costos de electricidad/gas) y los no business no es tan relevante
# clima historical vs forecast , adaptar columnas que usan distintas proporciones , medir la certeza del forecast
# analizar por fuera como funcionan los paneles y como miden lo que miden ( para mejor entendimiento del df)

#columnas a agregar
#Datetime dias lu-1/ma-2/mi-3/ju-4/vi-5/sa-6/do-7 
#Date time horas y capaz minutos
#horario laboral ejemplo 7 a 17hs (analizarlo) sea en el dataset y por fuera
#horario business , para los que son business ver un pseudo horario de apertura y cierre como afecta y si se puede
#Dia 6hs a 18hs / noche de 18hs a 6hs ver si lo adapto con 4 columnas por las estaciones
#hora de dormir ejemplo 22hs a 6hs
#dias festivos y vacaciones , columnas binarias en ambos
#estaciones del año 4 columnas binarias
#periodogram para ver los lags y sea mensuales/quincenales/diarios/hora
#ver si es relevante agregar alguna columna con la poblacion mensual y usar un ratio (info a buscar afuera)

#sugerencias de features

##Energéticas:

# Ratio production/consumption por prediction_unit_id (histórico)
# Capacidad instalada per capita (installed_capacity / eic_count)
# Eficiencia de paneles (production vs solar_radiation cuando is_consumption=False)
# Volatilidad de precios (rolling std de electricity/gas prices)
# Spread gas-electricidad (diferencia de precios, importante para switching)

##Temporales:

# Lag de consumo/producción del mismo prediction_unit_id (24h, 48h, 168h)
# Media móvil de target por prediction_unit_id
# Cambios día a día (delta vs día anterior)

##Weather engineering:

# Índice de confort térmico (combinando temp + humidity)
# Potencial solar real (direct + diffuse radiation ajustado por cloudcover)
# Diferencia forecast vs historical weather (para medir accuracy del forecast)

##Segmentación:

# Clustering de prediction_units por comportamiento similar
# Ratios county-level (agregaciones por county vs individual)


In [5]:
# hacer scatterplot con 
# train3.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1) el alpha para ver con mas claridad las zonas mas poblada
# podria agregar colores para diferencia temperatura o exposicion a rayos solares o nubes y poder encarar distintos el dataset
# sino funciona probar %matplotlib inline
# train3.hist(bins=50, figsize=(20,15)) ver tmb los histograma de los datos cuando este todo junto# cambiar los bins a valores mas chicos
# plt.show() agregar si hace falta
# corr_matrix = train3.corr()
# corr_matrix["target"].sort_values(ascending=False) ver si aplica pero creo que no , recordad probar en columnas numericas si hay otra relacion
# ejemplo crear columna con raiz/potencia/log y ver como apartan esos valores en correlacion
# o usar scatter_matrix() de pandas

In [6]:
train                 = pd.read_csv("/kaggle/input/predict-energy-behavior-of-prosumers/train.csv")
client                = pd.read_csv('/kaggle/input/predict-energy-behavior-of-prosumers/client.csv')
electricity_prices    = pd.read_csv('/kaggle/input/predict-energy-behavior-of-prosumers/electricity_prices.csv')
forecast_weather      = pd.read_csv('/kaggle/input/predict-energy-behavior-of-prosumers/forecast_weather.csv')
gas_prices            = pd.read_csv('/kaggle/input/predict-energy-behavior-of-prosumers/gas_prices.csv')
historical_weather    = pd.read_csv('/kaggle/input/predict-energy-behavior-of-prosumers/historical_weather.csv')
weather_station       = pd.read_csv('/kaggle/input/predict-energy-behavior-of-prosumers/weather_station_to_county_mapping.csv')

test                  = pd.read_csv('/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/test.csv')
client_t              = pd.read_csv('/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/client.csv')
electricity_prices_t  = pd.read_csv('/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/electricity_prices.csv')
forecast_weather_t    = pd.read_csv('/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/forecast_weather.csv')
gas_prices_t          = pd.read_csv('/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/gas_prices.csv')
revealed_targets      = pd.read_csv('/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/revealed_targets.csv')


In [7]:
# pruebo pipeline

In [8]:
import pandas as pd
import numpy as np
import unicodedata
from sklearn.neighbors import NearestNeighbors
import gc
from collections import defaultdict
from pathlib import Path

class DataPreprocessingPipeline:
    """Pipeline modular para preprocessing de datos de train y test"""
    
    def __init__(self):
        self.county_id_to_name_map = {
            0: "HARJUMAA", 1: "HIIUMAA", 2: "IDA-VIRUMAA", 3: "JÄRVAMAA",
            4: "JÕGEVAMAA", 5: "LÄÄNE-VIRUMAA", 6: "LÄÄNEMAA", 7: "PÄRNUMAA",
            8: "PÕLVAMAA", 9: "RAPLAMAA", 10: "SAAREMAA", 11: "TARTUMAA",
            12: "UNKNOWN", 13: "VALGAMAA", 14: "VILJANDIMAA", 15: "VÕRUMAA"
        }
        self.county_name_to_id_map = {v: k for k, v in self.county_id_to_name_map.items()}
        
        # Diccionario de sinónimos para normalización
        self.synonyms_to_canonical = {
            "HARJU": "HARJUMAA", "HIIU": "HIIUMAA", "IDA-VIRU": "IDA-VIRUMAA",
            "JARVA": "JÄRVAMAA", "JÄRVA": "JÄRVAMAA", "JOGEVA": "JÕGEVAMAA", 
            "JÕGEVA": "JÕGEVAMAA", "LAANE-VIRU": "LÄÄNE-VIRUMAA", 
            "LÄÄNE-VIRU": "LÄÄNE-VIRUMAA", "LAANE": "LÄÄNEMAA", "LÄÄNE": "LÄÄNEMAA",
            "PARNU": "PÄRNUMAA", "PÄRNU": "PÄRNUMAA", "POLVA": "PÕLVAMAA", 
            "PÕLVA": "PÕLVAMAA", "RAPLA": "RAPLAMAA", "SAARE": "SAAREMAA",
            "TARTU": "TARTUMAA", "VALGA": "VALGAMAA", "VILJANDI": "VILJANDIMAA",
            "VORU": "VÕRUMAA", "VÕRU": "VÕRUMAA",
            # versiones ya canónicas
            **{name: name for name in self.county_id_to_name_map.values()}
        }
    
    def strip_accents(self, s: str) -> str:
        """Eliminar acentos de string"""
        return ''.join(c for c in unicodedata.normalize('NFKD', s) if not unicodedata.combining(c))
    
    def normalize_county_name(self, x):
        """Normalizar nombre de county"""
        if pd.isna(x):
            return np.nan
        s = str(x).upper().strip()
        # eliminar sufijos frecuentes
        for suf in [" COUNTY", " MAAKOND"]:
            if s.endswith(suf):
                s = s[: -len(suf)]
        s = s.replace("  ", " ").replace("–", "-")
        s_noacc = self.strip_accents(s)
        s_noacc = s_noacc.replace("  ", " ")
        
        token = s_noacc.split()[0]
        if "-" in s_noacc:
            token = s_noacc
        
        # intentos de match
        if s_noacc in self.synonyms_to_canonical:
            return self.synonyms_to_canonical[s_noacc]
        if token in self.synonyms_to_canonical:
            return self.synonyms_to_canonical[token]
        if s in self.synonyms_to_canonical:
            return self.synonyms_to_canonical[s]
        
        return np.nan
    
    def process_weather_stations(self, weather_station):
        """Procesar estaciones meteorológicas para asignar counties"""
        weather_station = weather_station.copy()
        
        # Normalizar nombres de county
        weather_station['county_name_norm'] = weather_station['county_name'].apply(self.normalize_county_name)
        
        # Si tengo 'county' numérico pero falta nombre, lo inferimos del ID
        mask = weather_station['county_name_norm'].isna() & weather_station['county'].notna()
        weather_station.loc[mask, 'county_name_norm'] = (
            weather_station.loc[mask, 'county'].astype(int).map(self.county_id_to_name_map)
        )
        
        # Completar faltantes por k-NN
        fcols = ['latitude', 'longitude']
        known = weather_station.dropna(subset=['county_name_norm']).copy()
        unknown = weather_station[weather_station['county_name_norm'].isna()].copy()
        
        if not unknown.empty and not known.empty:
            known_rad = np.radians(known[fcols].values)
            unknown_rad = np.radians(unknown[fcols].values)
            
            nbrs = NearestNeighbors(n_neighbors=3, metric='haversine')
            nbrs.fit(known_rad)
            distances, idxs = nbrs.kneighbors(unknown_rad)
            
            nn_names = known['county_name_norm'].values
            filled = []
            for neigh_idx in idxs:
                cands = nn_names[neigh_idx]
                vals, counts = np.unique(cands, return_counts=True)
                filled.append(vals[np.argmax(counts)])
            
            weather_station.loc[unknown.index, 'county_name_norm'] = filled
        
        # Sincronizar columnas finales
        weather_station['county_name'] = weather_station['county_name_norm']
        weather_station['county'] = weather_station['county_name'].map(self.county_name_to_id_map).astype('Int64')
        
        # Cualquier remanente a UNKNOWN (12)
        weather_station.loc[weather_station['county_name'].isna(), 'county'] = 12
        weather_station['county'] = weather_station['county'].astype('Int64')
        
        weather_station.drop(columns=['county_name_norm'], inplace=True)
        
        return weather_station
    
    def optimize_dtypes(self, df, is_train=True):
        """Optimizar tipos de datos para reducir memoria"""
        df = df.copy()
        
        # Tipos comunes
        if 'county' in df.columns:
            df['county'] = df['county'].astype('uint8')
        if 'is_business' in df.columns:
            df['is_business'] = df['is_business'].astype('uint8')
        if 'product_type' in df.columns:
            df['product_type'] = df['product_type'].astype('uint8')
        if 'is_consumption' in df.columns:
            df['is_consumption'] = df['is_consumption'].astype('uint8')
        if 'hour' in df.columns:
            df['hour'] = df['hour'].astype('uint8')
        
        # IDs
        if 'data_block_id' in df.columns:
            df['data_block_id'] = df['data_block_id'].astype('uint16')
        if 'row_id' in df.columns:
            df['row_id'] = df['row_id'].astype('uint32')
        if 'prediction_unit_id' in df.columns:
            df['prediction_unit_id'] = df['prediction_unit_id'].astype('uint8')
        
        # Coordenadas y variables meteorológicas - float32
        float_cols = ['latitude', 'longitude', 'target', 'lowest_price_per_mwh', 
                     'highest_price_per_mwh', 'euros_per_mwh', 'eic_count', 
                     'installed_capacity']
        
        # Variables meteorológicas
        weather_cols = ['ftemperature', 'fdewpoint', 'fcloudcover_high', 'fcloudcover_low', 
                       'fcloudcover_mid', 'fcloudcover_total', 'f10_metre_u_wind_component', 
                       'f10_metre_v_wind_component', 'fdirect_solar_radiation', 
                       'fsurface_solar_radiation_downwards', 'fsnowfall', 'ftotal_precipitation',
                       'temperature', 'dewpoint', 'rain', 'snowfall', 'surface_pressure',
                       'cloudcover_total', 'cloudcover_low', 'cloudcover_mid', 'cloudcover_high',
                       'windspeed_10m', 'winddirection_10m', 'shortwave_radiation', 
                       'direct_solar_radiation', 'diffuse_radiation']
        
        # Aplicar float32 a columnas que existen
        for col in float_cols + weather_cols:
            if col in df.columns:
                df[col] = df[col].astype('float32')
        
        # Weather forecast hour
        if 'weather_forecast_hour' in df.columns:
            df['weather_forecast_hour'] = df['weather_forecast_hour'].astype('uint8')
        
        return df


class TrainPipeline(DataPreprocessingPipeline):
    """Pipeline específico para datos de entrenamiento"""
    
    def __init__(self):
        super().__init__()
        self.temp_dir = Path("temp_chunks")
        self.temp_dir.mkdir(exist_ok=True)
    
    def part1_prepare_base_merges(self, train, gas_prices, electricity_prices, client):
        """Parte 1: Preparar datos base y hacer primeros merges"""
        print("=== PARTE 1: Preparación y merges base ===")
        
        # Preparar copias
        train1 = train.dropna().copy()
        gas_prices1 = gas_prices.copy()
        electricity_prices1 = electricity_prices.copy()
        client1 = client.copy()
        
        # Procesar train
        train1['datetime'] = pd.to_datetime(train1['datetime'])
        train1['hour'] = train1['datetime'].dt.hour
        train1['forecast_date'] = train1['datetime']
        
        # Merge 1: Gas prices
        print("Procesando gas prices...")
        gas_prices1['origin_date'] = pd.to_datetime(gas_prices1['origin_date'])
        gas_prices1['forecast_date'] = pd.to_datetime(gas_prices1['forecast_date'])
        gas_prices1 = gas_prices1.rename(columns={'origin_date': 'gas_origin_date'})
        
        # Expandir gas a nivel horario
        gas_hourly = []
        for _, row in gas_prices1.iterrows():
            for hour in range(24):
                new_row = {
                    'forecast_date': pd.to_datetime(row['forecast_date']) + pd.Timedelta(hours=hour),
                    'lowest_price_per_mwh': row['lowest_price_per_mwh'],
                    'highest_price_per_mwh': row['highest_price_per_mwh']
                }
                gas_hourly.append(new_row)
        
        gas_prices1_hourly = pd.DataFrame(gas_hourly)
        train2 = pd.merge(train1, gas_prices1_hourly, on='forecast_date', how='left')
        
        # Forward fill gas prices
        train2['lowest_price_per_mwh'] = train2['lowest_price_per_mwh'].fillna(method='ffill')
        train2['highest_price_per_mwh'] = train2['highest_price_per_mwh'].fillna(method='ffill')
        
        # Merge 2: Electricity prices
        print("Procesando electricity prices...")
        electricity_prices1['forecast_date'] = pd.to_datetime(electricity_prices1['forecast_date'])
        electricity_prices1 = electricity_prices1.drop(columns=['origin_date'])
        
        elec_columns = ['forecast_date', 'euros_per_mwh']
        train3 = pd.merge(train2, electricity_prices1[elec_columns], on='forecast_date', how='left')
        train3['euros_per_mwh'] = train3['euros_per_mwh'].fillna(method='ffill')
        
        # Merge 3: Client data
        print("Procesando client data...")
        client1['date'] = pd.to_datetime(client1['date']).dt.date
        
        train4 = train3.merge(client1.drop('data_block_id', axis=1),
                             left_on=['product_type', 'county', 'is_business', train3['datetime'].dt.date],
                             right_on=['product_type', 'county', 'is_business', 'date'],
                             how='left')
        
        train4 = train4.drop('date', axis=1)
        train4 = train4.rename(columns={'data_block_id_x': 'data_block_id'})
        
        # Forward fill client data
        print("Completando datos de clientes...")
        train4 = train4.sort_values(['county', 'is_business', 'product_type', 'datetime'])
        train4['eic_count'] = train4.groupby(['county', 'is_business', 'product_type'])['eic_count'].fillna(method='ffill')
        train4['installed_capacity'] = train4.groupby(['county', 'is_business', 'product_type'])['installed_capacity'].fillna(method='ffill')
        
        # Rellenar restantes con promedio por county
        train4['eic_count'] = train4['eic_count'].fillna(train4.groupby('county')['eic_count'].transform('mean'))
        train4['installed_capacity'] = train4['installed_capacity'].fillna(train4.groupby('county')['installed_capacity'].transform('mean'))
        
        # Optimizar tipos
        train4 = self.optimize_dtypes(train4, is_train=True)
        
        print(f"Parte 1 completada. Dataset: {train4.shape[0]:,} filas, {train4.shape[1]} columnas")
        return train4
    
    def part2_prepare_weather_merge(self, train4, forecast_weather, historical_weather, weather_station):
        """Parte 2: Preparar datos meteorológicos y hacer merge"""
        print("=== PARTE 2: Preparación datos meteorológicos ===")
        
        # Procesar weather stations
        weather_station_processed = self.process_weather_stations(weather_station)
        
        # Procesar forecast weather
        print("Procesando forecast weather...")
        forecast_weather1 = forecast_weather.dropna().copy()
        
        # Merge forecast con weather stations
        forecast_weather1 = forecast_weather1.merge(
            weather_station_processed[['latitude', 'longitude', 'county', 'county_name']], 
            on=['latitude', 'longitude'], 
            how='left'
        )
        
        # Completar counties faltantes con k-NN
        missing_county = forecast_weather1['county'].isna()
        if missing_county.sum() > 0:
            print(f"Completando {missing_county.sum()} counties faltantes con k-NN...")
            known = weather_station_processed[['latitude', 'longitude', 'county', 'county_name']].copy()
            missing_coords = forecast_weather1.loc[missing_county, ['latitude', 'longitude']].copy()
            
            nbrs = NearestNeighbors(n_neighbors=1, metric='euclidean')
            nbrs.fit(known[['latitude', 'longitude']].values)
            distances, indices = nbrs.kneighbors(missing_coords[['latitude', 'longitude']].values)
            
            nearest_counties = known.iloc[indices.flatten()]
            forecast_weather1.loc[missing_county, 'county'] = nearest_counties['county'].values
            forecast_weather1.loc[missing_county, 'county_name'] = nearest_counties['county_name'].values
        
        # Transformar forecast weather
        forecast_weather2 = forecast_weather1.copy()
        forecast_weather2 = forecast_weather2.drop(['county_name', 'origin_datetime'], axis=1)
        forecast_weather2['forecast_datetime'] = pd.to_datetime(forecast_weather2['forecast_datetime'])
        
        # Renombrar con prefijo 'f'
        rename_dict = {}
        for col in forecast_weather2.columns:
            if col not in ['county', 'hours_ahead', 'latitude', 'longitude', 'data_block_id', 
                          'forecast_datetime']:
                rename_dict[col] = 'f' + col
        
        forecast_weather2 = forecast_weather2.rename(columns=rename_dict)
        forecast_weather2 = forecast_weather2.rename(columns={"hours_ahead": "weather_forecast_hour"})
        
        # Merge con historical weather
        print("Mergeando forecast con historical weather...")
        historical_weather1 = historical_weather.copy()
        historical_weather1['datetime'] = pd.to_datetime(historical_weather1['datetime'])
        
        forecast_weather2 = forecast_weather2.rename(columns={'data_block_id': 'f_data_block_id'})
        
        merged_weather = pd.merge(
            forecast_weather2,
            historical_weather1,
            left_on=['latitude', 'longitude', 'forecast_datetime'],
            right_on=['latitude', 'longitude', 'datetime'],
            how='inner'
        )
        
        # Limpiar y optimizar
        merged_weather = merged_weather.drop(['f_data_block_id', 'forecast_datetime'], axis=1)
        merged_weather = self.optimize_dtypes(merged_weather, is_train=True)
        
        print(f"Weather data preparado: {merged_weather.shape[0]:,} filas, {merged_weather.shape[1]} columnas")
        return train4, merged_weather
    
    def part3_final_merge_and_cleanup(self, train4, merged_weather, chunk_size=1_000_000):
        """Parte 3: Merge final y limpieza"""
        print("=== PARTE 3: Merge final y limpieza ===")
        
        # Definir períodos para procesar por chunks
        periods = [
            ('2021-09-01', '2021-10-01'), ('2021-10-01', '2021-11-01'), ('2021-11-01', '2021-12-01'), 
            ('2021-12-01', '2022-01-01'), ('2022-01-01', '2022-02-01'), ('2022-02-01', '2022-03-01'), 
            ('2022-03-01', '2022-04-01'), ('2022-04-01', '2022-05-01'), ('2022-05-01', '2022-06-01'), 
            ('2022-06-01', '2022-07-01'), ('2022-07-01', '2022-08-01'), ('2022-08-01', '2022-09-01'),
            ('2022-09-01', '2022-10-01'), ('2022-10-01', '2022-11-01'), ('2022-11-01', '2022-12-01'), 
            ('2022-12-01', '2023-01-01'), ('2023-01-01', '2023-02-01'), ('2023-02-01', '2023-03-01'), 
            ('2023-03-01', '2023-04-01'), ('2023-04-01', '2023-05-01'), ('2023-05-01', '2023-05-31')
        ]
        
        # Procesar por chunks temporales
        chunk_files = []
        for i, (start_date, end_date) in enumerate(periods):
            print(f"Procesando período {i+1}/{len(periods)}: {start_date} a {end_date}")
            
            # Filtrar por período
            mask_train = (train4['datetime'] >= start_date) & (train4['datetime'] < end_date)
            train4_chunk = train4[mask_train].copy()
            
            mask_weather = (merged_weather['datetime'] >= start_date) & (merged_weather['datetime'] < end_date)
            weather_chunk = merged_weather[mask_weather].copy()
            
            if len(train4_chunk) > 0 and len(weather_chunk) > 0:
                # Merge chunk
                chunk_result = pd.merge(train4_chunk, weather_chunk, 
                               on=['county', 'datetime'], 
                               how='left')
                
                # Guardar chunk temporal
                chunk_file = self.temp_dir / f'train5_chunk_{i+1}.parquet'
                chunk_result.to_parquet(chunk_file, index=False)
                chunk_files.append(chunk_file)
                
                print(f"Chunk {i+1} guardado: {chunk_result.shape}")
                
                # Liberar memoria
                del train4_chunk, weather_chunk, chunk_result
                gc.collect()
        
        # Concatenar chunks
        print("Concatenando chunks...")
        final_chunks = []
        for chunk_file in chunk_files:
            chunk_data = pd.read_parquet(chunk_file)
            final_chunks.append(chunk_data)
        
        train5 = pd.concat(final_chunks, ignore_index=True)
        
        # Limpieza final
        print("Aplicando limpieza final...")
        train5 = self._cleanup_merged_data(train5, chunk_size)
        
        # Limpiar archivos temporales
        for chunk_file in chunk_files:
            chunk_file.unlink()
        
        return train5
    
    def _cleanup_merged_data(self, train5, chunk_size=1_000_000):
        """Limpiar datos después del merge"""
        # Identificar columnas a eliminar y renombrar
        sample = train5.head(1000)
        cols_to_drop = [col for col in sample.columns if col.endswith('_y')]
        rename_dict = {col: col[:-2] for col in sample.columns if col.endswith('_x')}
        
        print(f"Eliminando {len(cols_to_drop)} columnas, renombrando {len(rename_dict)}")
        
        # Procesar por chunks
        chunks_processed = []
        total_rows = len(train5)
        n_chunks = (total_rows // chunk_size) + 1
        
        for i in range(n_chunks):
            start_idx = i * chunk_size
            end_idx = min((i + 1) * chunk_size, total_rows)
            
            print(f"Procesando chunk limpieza {i+1}/{n_chunks}: filas {start_idx:,} a {end_idx:,}")
            
            chunk = train5.iloc[start_idx:end_idx].copy()
            
            # Filtrar county != 12
            chunk = chunk[chunk['county'] != 12]
            
            if len(chunk) > 0:
                # Eliminar columnas _y y renombrar _x
                chunk = chunk.drop(columns=cols_to_drop)
                chunk.rename(columns=rename_dict, inplace=True)
                chunks_processed.append(chunk)
            
            del chunk
            gc.collect()
        
        # Concatenar final
        train5_clean = pd.concat(chunks_processed, ignore_index=True)
        del chunks_processed
        gc.collect()
        
        # Eliminar datos de clima faltantes
        weather_key_cols = ['latitude', 'longitude', 'ftemperature', 'temperature']
        initial_rows = len(train5_clean)
        train5_clean = train5_clean.dropna(subset=weather_key_cols)
        print(f"Filas eliminadas por clima faltante: {initial_rows - len(train5_clean):,}")
        
        # Reordenar columnas
        train5_clean = self._reorder_columns(train5_clean)
        
        return train5_clean
    
    def _reorder_columns(self, df):
        """Reordenar columnas en el orden especificado"""
        main_cols = [
            'row_id', 'target', 'datetime', 'forecast_date', 'hour', 'data_block_id', 
            'prediction_unit_id', 'is_business', 'product_type', 'county', 'latitude', 
            'longitude', 'lowest_price_per_mwh', 'highest_price_per_mwh', 'euros_per_mwh', 
            'eic_count', 'installed_capacity'
        ]
        
        weather_pairs = [
            'ftemperature', 'temperature', 'fdewpoint', 'dewpoint',
            'fcloudcover_high', 'cloudcover_high', 'fcloudcover_low', 'cloudcover_low', 
            'fcloudcover_mid', 'cloudcover_mid', 'fcloudcover_total', 'cloudcover_total',
            'f10_metre_u_wind_component', 'windspeed_10m', 'f10_metre_v_wind_component', 
            'winddirection_10m', 'fdirect_solar_radiation', 'direct_solar_radiation',
            'fsurface_solar_radiation_downwards', 'shortwave_radiation', 'diffuse_radiation',
            'fsnowfall', 'snowfall', 'ftotal_precipitation', 'rain', 'surface_pressure', 
            'weather_forecast_hour'
        ]
        
        final_order = main_cols.copy()
        for col in weather_pairs:
            if col in df.columns:
                final_order.append(col)
        
        remaining_cols = [col for col in df.columns if col not in final_order]
        final_order.extend(remaining_cols)
        
        return df[final_order]
    
    def run_full_pipeline(self, train, gas_prices, electricity_prices, client, 
                         forecast_weather, historical_weather, weather_station):
        """Ejecutar pipeline completo de entrenamiento"""
        print("Iniciando pipeline completo de entrenamiento...")
        
        # Parte 1: Merges base
        train4 = self.part1_prepare_base_merges(train, gas_prices, electricity_prices, client)
        
        # Parte 2: Preparar datos meteorológicos
        train4, merged_weather = self.part2_prepare_weather_merge(
            train4, forecast_weather, historical_weather, weather_station
        )
        
        # Parte 3: Merge final y limpieza
        train5 = self.part3_final_merge_and_cleanup(train4, merged_weather)
        
        print(f"Pipeline completado! Dataset final: {train5.shape[0]:,} filas, {train5.shape[1]} columnas")
        return train5


class TestPipeline(DataPreprocessingPipeline):
    """Pipeline simplificado para datos de test"""
    
    def run_test_pipeline(self, test, gas_prices_t, electricity_prices_t, client_t, 
                         forecast_weather_t, weather_station):
        """Pipeline completo para test set (más simple, sin historical weather)"""
        print("=== PIPELINE TEST ===")
        
        # Preparar datos base
        test1 = test.copy()
        test1['datetime'] = pd.to_datetime(test1['prediction_datetime'])
        test1['hour'] = test1['datetime'].dt.hour
        test1['forecast_date'] = test1['datetime']
        
        # Merge gas prices
        print("Procesando gas prices...")
        gas_prices1 = gas_prices_t.copy()
        gas_prices1['origin_date'] = pd.to_datetime(gas_prices1['origin_date'])
        gas_prices1['forecast_date'] = pd.to_datetime(gas_prices1['forecast_date'])
        
        gas_hourly = []
        for _, row in gas_prices1.iterrows():
            for hour in range(24):
                new_row = {
                    'forecast_date': pd.to_datetime(row['forecast_date']) + pd.Timedelta(hours=hour),
                    'lowest_price_per_mwh': row['lowest_price_per_mwh'],
                    'highest_price_per_mwh': row['highest_price_per_mwh']
                }
                gas_hourly.append(new_row)
        
        gas_prices_hourly = pd.DataFrame(gas_hourly)
        test2 = pd.merge(test1, gas_prices_hourly, on='forecast_date', how='left')
        test2['lowest_price_per_mwh'] = test2['lowest_price_per_mwh'].fillna(method='ffill')
        test2['highest_price_per_mwh'] = test2['highest_price_per_mwh'].fillna(method='ffill')
        
        # Merge electricity prices
        print("Procesando electricity prices...")
        electricity_prices1 = electricity_prices_t.copy()
        electricity_prices1['forecast_date'] = pd.to_datetime(electricity_prices1['forecast_date'])
        electricity_prices1 = electricity_prices1.drop(columns=['origin_date'])
        
        test3 = pd.merge(test2, electricity_prices1[['forecast_date', 'euros_per_mwh']], 
                        on='forecast_date', how='left')
        test3['euros_per_mwh'] = test3['euros_per_mwh'].fillna(method='ffill')
        
        # Merge client data
        print("Procesando client data...")
        client1 = client_t.copy()
        client1['date'] = pd.to_datetime(client1['date']).dt.date
        
        test4 = test3.merge(client1.drop('data_block_id', axis=1),
                           left_on=['product_type', 'county', 'is_business', test3['datetime'].dt.date],
                           right_on=['product_type', 'county', 'is_business', 'date'],
                           how='left')
        
        test4 = test4.drop('date', axis=1)
        test4 = test4.rename(columns={'data_block_id_x': 'data_block_id'})
        
        # Forward fill client data
        test4 = test4.sort_values(['county', 'is_business', 'product_type', 'datetime'])
        test4['eic_count'] = test4.groupby(['county', 'is_business', 'product_type'])['eic_count'].fillna(method='ffill')
        test4['installed_capacity'] = test4.groupby(['county', 'is_business', 'product_type'])['installed_capacity'].fillna(method='ffill')
        
        # Rellenar restantes con promedio por county
        test4['eic_count'] = test4['eic_count'].fillna(test4.groupby('county')['eic_count'].transform('mean'))
        test4['installed_capacity'] = test4['installed_capacity'].fillna(test4.groupby('county')['installed_capacity'].transform('mean'))
        
        # Procesar forecast weather
        print("Procesando forecast weather...")
        weather_station_processed = self.process_weather_stations(weather_station)
        
        forecast_weather1 = forecast_weather_t.copy()
        
        # Merge forecast con weather stations
        forecast_weather1 = forecast_weather1.merge(
            weather_station_processed[['latitude', 'longitude', 'county', 'county_name']], 
            on=['latitude', 'longitude'], 
            how='left'
        )
        
        # Completar counties faltantes con k-NN
        missing_county = forecast_weather1['county'].isna()
        if missing_county.sum() > 0:
            print(f"Completando {missing_county.sum()} counties faltantes con k-NN...")
            known = weather_station_processed[['latitude', 'longitude', 'county', 'county_name']].copy()
            missing_coords = forecast_weather1.loc[missing_county, ['latitude', 'longitude']].copy()
            
            nbrs = NearestNeighbors(n_neighbors=1, metric='euclidean')
            nbrs.fit(known[['latitude', 'longitude']].values)
            distances, indices = nbrs.kneighbors(missing_coords[['latitude', 'longitude']].values)
            
            nearest_counties = known.iloc[indices.flatten()]
            forecast_weather1.loc[missing_county, 'county'] = nearest_counties['county'].values
            forecast_weather1.loc[missing_county, 'county_name'] = nearest_counties['county_name'].values
        
        # Transformar forecast weather
        forecast_weather2 = forecast_weather1.copy()
        forecast_weather2 = forecast_weather2.drop(['county_name', 'origin_datetime'], axis=1)
        forecast_weather2['forecast_datetime'] = pd.to_datetime(forecast_weather2['forecast_datetime'])
        
        # Renombrar con prefijo 'f'
        rename_dict = {}
        for col in forecast_weather2.columns:
            if col not in ['county', 'hours_ahead', 'latitude', 'longitude', 'data_block_id', 
                          'forecast_datetime']:
                rename_dict[col] = 'f' + col
        
        forecast_weather2 = forecast_weather2.rename(columns=rename_dict)
        forecast_weather2 = forecast_weather2.rename(columns={"hours_ahead": "weather_forecast_hour"})
        
        # Para test, no tenemos historical weather, así que creamos columnas dummy o usamos solo forecast
        # Opción 1: Solo usar forecast weather (renombrar las f columns)
        # Opción 2: Crear columnas históricas como NaN y llenar después
        
        # Vamos con opción 1: usar forecast como histórico también (aproximación)
        historical_cols_mapping = {
            'ftemperature': 'temperature',
            'fdewpoint': 'dewpoint', 
            'fcloudcover_high': 'cloudcover_high',
            'fcloudcover_low': 'cloudcover_low',
            'fcloudcover_mid': 'cloudcover_mid', 
            'fcloudcover_total': 'cloudcover_total',
            'fdirect_solar_radiation': 'direct_solar_radiation',
            'fsurface_solar_radiation_downwards': 'shortwave_radiation',
            'fsnowfall': 'snowfall',
            'ftotal_precipitation': 'rain'
        }
        
        # Crear versiones históricas basadas en forecast
        for fcol, hcol in historical_cols_mapping.items():
            if fcol in forecast_weather2.columns:
                forecast_weather2[hcol] = forecast_weather2[fcol]
        
        # Agregar columnas que solo existen en historical
        forecast_weather2['surface_pressure'] = 1013.25  # valor típico
        forecast_weather2['windspeed_10m'] = np.sqrt(
            forecast_weather2['f10_metre_u_wind_component']**2 + 
            forecast_weather2['f10_metre_v_wind_component']**2
        ) if 'f10_metre_u_wind_component' in forecast_weather2.columns else 0
        
        forecast_weather2['winddirection_10m'] = np.arctan2(
            forecast_weather2['f10_metre_v_wind_component'], 
            forecast_weather2['f10_metre_u_wind_component']
        ) * 180 / np.pi if 'f10_metre_v_wind_component' in forecast_weather2.columns else 0
        
        forecast_weather2['diffuse_radiation'] = forecast_weather2.get('fsurface_solar_radiation_downwards', 0) - forecast_weather2.get('fdirect_solar_radiation', 0)
        forecast_weather2['diffuse_radiation'] = forecast_weather2['diffuse_radiation'].clip(lower=0)
        
        # Limpiar y optimizar
        forecast_weather2 = forecast_weather2.drop(columns=['data_block_id'], errors='ignore')
        forecast_weather2 = self.optimize_dtypes(forecast_weather2, is_train=False)
        
        # Merge final con weather
        print("Merge final con datos meteorológicos...")
        test5 = pd.merge(test4, forecast_weather2, 
                        left_on=['datetime', 'county'], 
                        right_on=['forecast_datetime', 'county'], 
                        how='left')



        # Limpiar columnas duplicadas y renombrar
        cols_to_drop = [col for col in test5.columns if col.endswith('_y')]
        rename_dict = {col: col[:-2] for col in test5.columns if col.endswith('_x')}
        
        test5 = test5.drop(columns=cols_to_drop + ['forecast_datetime'], errors='ignore')
        test5.rename(columns=rename_dict, inplace=True)
        
        # Optimizar tipos finales
        test5 = self.optimize_dtypes(test5, is_train=False)
        
        # Reordenar columnas igual que train
        test5 = self._reorder_columns_test(test5)
        
        print(f"Pipeline test completado! Dataset: {test5.shape[0]:,} filas, {test5.shape[1]} columnas")
        return test5
    
    def _reorder_columns_test(self, df):
        """Reordenar columnas para test (mismo orden que train)"""
        main_cols = [
            'row_id', 'datetime', 'forecast_date', 'hour', 'data_block_id', 
            'prediction_unit_id', 'is_business', 'product_type', 'county', 'latitude', 
            'longitude', 'lowest_price_per_mwh', 'highest_price_per_mwh', 'euros_per_mwh', 
            'eic_count', 'installed_capacity'
        ]
        
        weather_pairs = [
            'ftemperature', 'temperature', 'fdewpoint', 'dewpoint',
            'fcloudcover_high', 'cloudcover_high', 'fcloudcover_low', 'cloudcover_low', 
            'fcloudcover_mid', 'cloudcover_mid', 'fcloudcover_total', 'cloudcover_total',
            'f10_metre_u_wind_component', 'windspeed_10m', 'f10_metre_v_wind_component', 
            'winddirection_10m', 'fdirect_solar_radiation', 'direct_solar_radiation',
            'fsurface_solar_radiation_downwards', 'shortwave_radiation', 'diffuse_radiation',
            'fsnowfall', 'snowfall', 'ftotal_precipitation', 'rain', 'surface_pressure', 
            'weather_forecast_hour'
        ]
        
        final_order = []
        # Solo agregar columnas que existen
        for col in main_cols:
            if col in df.columns:
                final_order.append(col)
        
        for col in weather_pairs:
            if col in df.columns:
                final_order.append(col)
        
        remaining_cols = [col for col in df.columns if col not in final_order]
        final_order.extend(remaining_cols)
        
        return df[final_order]


# Funciones de utilidad para usar el pipeline
def run_train_pipeline(train, gas_prices, electricity_prices, client, 
                      forecast_weather, historical_weather, weather_station):
    """
    Función principal para ejecutar pipeline de entrenamiento
    
    Returns:
        pd.DataFrame: Dataset procesado train5
    """
    pipeline = TrainPipeline()
    return pipeline.run_full_pipeline(
        train, gas_prices, electricity_prices, client, 
        forecast_weather, historical_weather, weather_station
    )

def run_test_pipeline(test, gas_prices_t, electricity_prices_t, client_t, 
                     forecast_weather_t, weather_station):
    """
    Función principal para ejecutar pipeline de test
    
    Returns:
        pd.DataFrame: Dataset procesado test5
    """
    pipeline = TestPipeline()
    return pipeline.run_test_pipeline(
        test, gas_prices_t, electricity_prices_t, client_t, 
        forecast_weather_t, weather_station
    )

# Ejemplo de uso:
"""
# Para train:
train5 = run_train_pipeline(
    train, gas_prices, electricity_prices, client, 
    forecast_weather, historical_weather, weather_station
)

# Para test:
test5 = run_test_pipeline(
    test, gas_prices_t, electricity_prices_t, client_t, 
    forecast_weather_t, weather_station
)
"""

'\n# Para train:\ntrain5 = run_train_pipeline(\n    train, gas_prices, electricity_prices, client, \n    forecast_weather, historical_weather, weather_station\n)\n\n# Para test:\ntest5 = run_test_pipeline(\n    test, gas_prices_t, electricity_prices_t, client_t, \n    forecast_weather_t, weather_station\n)\n'

In [9]:
train5 = run_train_pipeline(
    train, gas_prices, electricity_prices, client, 
    forecast_weather, historical_weather, weather_station
)

Iniciando pipeline completo de entrenamiento...
=== PARTE 1: Preparación y merges base ===
Procesando gas prices...
Procesando electricity prices...
Procesando client data...
Completando datos de clientes...
Parte 1 completada. Dataset: 2,017,824 filas, 16 columnas
=== PARTE 2: Preparación datos meteorológicos ===
Procesando forecast weather...
Completando 2140318 counties faltantes con k-NN...
Mergeando forecast con historical weather...
Weather data preparado: 3,418,242 filas, 32 columnas
=== PARTE 3: Merge final y limpieza ===
Procesando período 1/21: 2021-09-01 a 2021-10-01
Chunk 1 guardado: (1423080, 46)
Procesando período 2/21: 2021-10-01 a 2021-11-01
Chunk 2 guardado: (1532066, 46)
Procesando período 3/21: 2021-11-01 a 2021-12-01
Chunk 3 guardado: (1484640, 46)
Procesando período 4/21: 2021-12-01 a 2022-01-01
Chunk 4 guardado: (1543056, 46)
Procesando período 5/21: 2022-01-01 a 2022-02-01
Chunk 5 guardado: (1590672, 46)
Procesando período 6/21: 2022-02-01 a 2022-03-01
Chunk 6 gu

In [10]:
train5.tail()

,row_id,target,datetime,forecast_date,hour,data_block_id,prediction_unit_id,is_business,product_type,county,latitude,longitude,lowest_price_per_mwh,highest_price_per_mwh,euros_per_mwh,eic_count,installed_capacity,ftemperature,temperature,fdewpoint,dewpoint,fcloudcover_high,cloudcover_high,fcloudcover_low,cloudcover_low,fcloudcover_mid,cloudcover_mid,fcloudcover_total,cloudcover_total,f10_metre_u_wind_component,windspeed_10m,f10_metre_v_wind_component,winddirection_10m,fdirect_solar_radiation,direct_solar_radiation,fsurface_solar_radiation_downwards,shortwave_radiation,diffuse_radiation,fsnowfall,snowfall,ftotal_precipitation,rain,surface_pressure,weather_forecast_hour,is_consumption
32965017,2013541,444.356995,2023-05-30 10:00:00,2023-05-30 10:00:00,10,636,60,1,3,15,57.599998,28.200001,29.0,34.0,82.370003,55.0,2188.199951,12.136377,13.8,2.330225,3.9,0.0,0.0,0.000000,7.0,0.966461,71.0,0.966461,49.0,-1.009770,2.055556,-0.927298,346.0,1.858164,155.0,191.511948,410.0,255.0,0.0,0.0,0.000000,0.1,1012.799988,8.0,1
32965018,2013541,444.356995,2023-05-30 10:00:00,2023-05-30 10:00:00,10,636,60,1,3,15,57.900002,26.700001,29.0,34.0,82.370003,55.0,2188.199951,14.534815,13.9,2.355859,3.3,0.0,0.0,0.270050,15.0,0.544281,73.0,0.652069,57.0,2.205562,2.916667,-1.008352,297.0,832.791504,249.0,685.467529,490.0,241.0,0.0,0.0,0.000056,0.1,1005.200012,8.0,1
32965019,2013541,444.356995,2023-05-30 10:00:00,2023-05-30 10:00:00,10,636,60,1,3,15,57.900002,27.200001,29.0,34.0,82.370003,55.0,2188.199951,13.503565,13.9,3.533350,2.9,0.0,0.0,0.024902,18.0,0.181091,66.0,0.184845,56.0,0.538569,3.194444,-1.362844,309.0,486.835938,257.0,500.649719,498.0,241.0,0.0,0.0,0.000000,0.0,1007.200012,8.0,1
32965020,2013541,444.356995,2023-05-30 10:00:00,2023-05-30 10:00:00,10,636,60,1,3,15,57.900002,27.700001,29.0,34.0,82.370003,55.0,2188.199951,13.150782,13.6,2.600976,3.2,0.0,0.0,0.000153,18.0,0.866364,66.0,0.866455,56.0,-0.223149,3.527778,-1.600393,317.0,26.035942,240.0,268.223053,482.0,242.0,0.0,0.0,0.000000,0.0,1011.099976,8.0,1
32965021,2013541,444.356995,2023-05-30 10:00:00,2023-05-30 10:00:00,10,636,60,1,3,15,57.900002,28.200001,29.0,34.0,82.370003,55.0,2188.199951,12.648584,13.6,1.409814,3.6,0.0,0.0,0.000000,9.0,0.997070,78.0,0.997070,55.0,-0.198003,3.194444,-2.190237,319.0,63.440388,186.0,304.685272,446.0,260.0,0.0,0.0,0.000000,0.0,1013.000000,8.0,1


In [11]:
# Features temporales
train5['day_of_week'] = train5['datetime'].dt.dayofweek
train5['month'] = train5['datetime'].dt.month
# Crear is_daylight (1 si hay radiación solar, 0 si es noche)
train5['is_daylight'] = (train5['direct_solar_radiation'] > 0).astype(int)
# Crear weekend (1 para sábado/domingo, 0 para lunes-viernes)
train5['weekend'] = (train5['day_of_week'] >= 5).astype(int)

In [12]:
#Mae puesto n1 kaggle 52.3090
#1er primer modelo xgb sobre train5 haciendo split 80/20 de train y test 
#Mae = 107.9896
#sin lag sin feature, sin nada , solo haciendo el merge


#agrego day of week y month para todos los siguientes

#2do modelo lgb 10% y sobre esto split 80/20 de train y test 
#Mae = 89.3

#3er modelo lgb 10% y sobre esto split 80/20 de train y test 
#Mae = 103.1249
#climate_vars = ['fsurface_solar_radiation_downwards', 'shortwave_radiation', 
#                'temperature', 'direct_solar_radiation']
# agrego lags de 1h/4h/12h en las variables de arriba
# empeoro porque agregue los lags al azar no hice un analisis para agregarlo en los marcos de tiempo relevante
# estas variables de lags las quito

#agrego is_daylight binaria

#4to modelo lgb 14.2% y sobre esto split 80/20 de train y test  
#de base solo agrego la variable is_daylight , pero ahora uso info de los ultimos 3 meses 14.2% aprox
# se que no deberia ser comparable al usar de base distinto , pero parte de aca ahora
#Mae = 86.6747

#agrego weekend binaria

#5to modelo lgb igual que el anterior , solo agregue binaria Weekend y no vi cambios
#Mae = 86.6747

#para todos los demas modelos ya tienen incluido
#day of week
#month
#is_daylight
#weekend

#6to modelo lgb , agregue variable binaria is_active_hours , horario de 8 a 17 hs binaria
#Mae = 90.4776 , empeoro el modelo la saco por ahora a is_active_hours

#7mo modelo lgb, agregue variable binaria is_sleeping_hours 22h-6h
#Mae = 98.3152 empeora el modelo , asi que  la saco a is_sleeping_hours

#empiezo a probar con variables
##Energéticas:

# Ratio production/consumption por prediction_unit_id (histórico)
# Capacidad instalada per capita (installed_capacity / eic_count)
# Eficiencia de paneles (production vs solar_radiation cuando is_consumption=False)
# Volatilidad de precios (rolling std de electricity/gas prices)
# Spread gas-electricidad (diferencia de precios, importante para switching)

#8vo modelo lgb agregue 3 variables energeticas capacity_per_obs, production_obs_ratio, consumption_obs_ratio
#Mae = 76.2703 mejor bastante el modelo

#9no modelo lgb mantengo las 3 anteriores y prueba una nueva Ratio production/cn

In [13]:
# 2do modelo lgb 10% y sobre esto split 80/20 de train y test 
#Mae = 89.3

# # Sample 10% manteniendo orden temporal
# train5 = train5.sort_values(['prediction_unit_id', 'datetime'])
# sample_df = train5.sample(frac=0.1, random_state=42).sort_values(['prediction_unit_id', 'datetime'])

# print(f"Dataset original: {len(train5):,}")
# print(f"Sample 10%: {len(sample_df):,}")

# # Convertir datetime
# sample_df['datetime'] = pd.to_datetime(sample_df['datetime'])
# sample_df['day_of_week'] = sample_df['datetime'].dt.dayofweek
# sample_df['month'] = sample_df['datetime'].dt.month

# # Features (todas menos datetime/forecast_date/target/row_id)
# feature_cols = [col for col in sample_df.columns 
#                 if col not in ['target', 'datetime', 'forecast_date', 'row_id']]

# # Split temporal 80/20
# split_idx = int(len(sample_df) * 0.8)
# train_data = sample_df.iloc[:split_idx]
# test_data = sample_df.iloc[split_idx:]

# X_train = train_data[feature_cols].dropna()
# y_train = train_data.loc[X_train.index, 'target']
# X_test = test_data[feature_cols].dropna()
# y_test = test_data.loc[X_test.index, 'target']

# # LightGBM (más eficiente en memoria)
# model = lgb.LGBMRegressor(
#     n_estimators=100,
#     max_depth=6,
#     learning_rate=0.1,
#     random_state=42,
#     n_jobs=-1,
#     verbose=-1
# )

# model.fit(X_train, y_train)

# # Resultados
# pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, pred)

# print(f"MAE: {mae:.4f}")
# print(f"Features usadas: {len(feature_cols)}")
# print(f"Train: {len(X_train):,} | Test: {len(X_test):,}")

# Dataset original: 32,963,024
# Sample 10%: 3,296,302
# MAE: 89.3026
# Features usadas: 43
# Train: 2,637,041 | Test: 659,261

In [14]:
# # Feature importance
# importance = model.feature_importances_
# feature_names = X_train.columns

# # Crear DataFrame y ordenar
# importance_df = pd.DataFrame({
#     'feature': feature_names,
#     'importance': importance
# }).sort_values('importance', ascending=False)

# # Ver top 15
# print("Top 15 features más importantes:")
# print(importance_df.head(15))

# # Plot opcional (si quieres visualizar)
# plt.figure(figsize=(10, 8))
# plt.barh(importance_df.head(15)['feature'], importance_df.head(15)['importance'])
# plt.title('Top 15 Feature Importance')
# plt.tight_layout()
# plt.show()

In [15]:
# importance_df['importance_pct'] = (importance_df['importance'] / importance_df['importance'].sum()) * 100
# print(importance_df.head(15)[['feature', 'importance', 'importance_pct']])

#                               feature  importance  importance_pct
# 40                      is_consumption         469       15.633333
# 12                  installed_capacity         436       14.533333
# 11                           eic_count         292        9.733333
# 2                   prediction_unit_id         228        7.600000
# 31  fsurface_solar_radiation_downwards         227        7.566667
# 0                                 hour         180        6.000000
# 1                        data_block_id         164        5.466667
# 41                         day_of_week         161        5.366667
# 32                 shortwave_radiation         111        3.700000
# 3                          is_business         105        3.500000
# 10                       euros_per_mwh          83        2.766667
# 14                         temperature          73        2.433333
# 42                               month          72        2.400000
# 30              direct_solar_radiation          60        2.000000
# 29             fdirect_solar_radiation          48        1.600000

In [16]:
#3er modelo lgb 10% y sobre esto split 80/20 de train y test 
# Mae = 103.1249
#climate_vars = ['fsurface_solar_radiation_downwards', 'shortwave_radiation', 
#                'temperature', 'direct_solar_radiation']
# agrego lags de 1h/4h/12h en las variables de arriba
# empeoro porque agregue los lags al azar no hice un analisis para agregarlo en los marcos de tiempo relevante
# Sample 10%




# train5 = train5.sort_values(['prediction_unit_id', 'datetime'])
# sample_df = train5.sample(frac=0.1, random_state=42).sort_values(['prediction_unit_id', 'datetime'])

# print(f"Sample 10%: {len(sample_df):,}")

# # Convertir datetime
# sample_df['datetime'] = pd.to_datetime(sample_df['datetime'])
# sample_df['day_of_week'] = sample_df['datetime'].dt.dayofweek
# sample_df['month'] = sample_df['datetime'].dt.month

# # Variables climáticas importantes para lags
# climate_vars = ['fsurface_solar_radiation_downwards', 'shortwave_radiation', 
#                 'temperature', 'direct_solar_radiation']

# # Crear lags por prediction_unit_id
# print("🔄 Creando lags climáticos...")
# lag_dfs = []

# for unit_id in sample_df['prediction_unit_id'].unique():
#     unit_data = sample_df[sample_df['prediction_unit_id'] == unit_id].copy()
    
#     # Lags de 1h, 4h, 12h para cada variable climática
#     for var in climate_vars:
#         if var in unit_data.columns:
#             unit_data[f'{var}_lag1h'] = unit_data[var].shift(1)
#             unit_data[f'{var}_lag4h'] = unit_data[var].shift(4)
#             unit_data[f'{var}_lag12h'] = unit_data[var].shift(12)
    
#     lag_dfs.append(unit_data)

# sample_with_lags = pd.concat(lag_dfs, ignore_index=True)
# sample_with_lags = sample_with_lags.sort_values(['prediction_unit_id', 'datetime'])

# print(f"Datos con lags: {len(sample_with_lags):,}")

# # Features (todas menos datetime/forecast_date/target/row_id)
# feature_cols = [col for col in sample_with_lags.columns 
#                 if col not in ['target', 'datetime', 'forecast_date', 'row_id']]

# # Split temporal
# split_idx = int(len(sample_with_lags) * 0.8)
# train_data = sample_with_lags.iloc[:split_idx]
# test_data = sample_with_lags.iloc[split_idx:]

# # Dropear NaN (por los lags)
# X_train = train_data[feature_cols].dropna()
# y_train = train_data.loc[X_train.index, 'target']
# X_test = test_data[feature_cols].dropna()
# y_test = test_data.loc[X_test.index, 'target']

# print(f"Features totales: {len(feature_cols)}")
# print(f"Train final: {len(X_train):,}")
# print(f"Test final: {len(X_test):,}")

# # LightGBM
# model = lgb.LGBMRegressor(
#     n_estimators=100,
#     max_depth=6,
#     learning_rate=0.1,
#     random_state=42,
#     n_jobs=-1,
#     verbose=-1
# )

# model.fit(X_train, y_train)

# # Resultados
# pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, pred)

# print(f"MAE con lags: {mae:.4f}")
# print(f"Mejora vs 89.3: {89.3 - mae:.1f}")

In [17]:
#

In [18]:
#4to modelo lgb de base solo agrego la variable is_daylight , pero ahora uso info de los ultimos 3 meses por memoria 
#Mae = 86.6747


# Preparar datos y tomar últimos 3 meses
# train5 = train5.sort_values(['prediction_unit_id', 'datetime'])
# train5['datetime'] = pd.to_datetime(train5['datetime'])

# # Últimos 3 meses
# max_date = train5['datetime'].max()
# cutoff_date = max_date - pd.DateOffset(months=3)
# sample_df = train5[train5['datetime'] >= cutoff_date].copy()

# print(f"Dataset original: {len(train5):,}")
# print(f"Últimos 3 meses: {len(sample_df):,}")
# print(f"Porcentaje: {len(sample_df)/len(train5)*100:.1f}%")

# # Features temporales
# sample_df['day_of_week'] = sample_df['datetime'].dt.dayofweek
# sample_df['month'] = sample_df['datetime'].dt.month

# # Crear is_daylight (1 si hay radiación solar, 0 si es noche)
# sample_df['is_daylight'] = (sample_df['direct_solar_radiation'] > 0).astype(int)

# print(f"Daylight distribution:")
# print(sample_df['is_daylight'].value_counts())

# # Features (todas menos datetime/forecast_date/target/row_id)
# feature_cols = [col for col in sample_df.columns 
#                 if col not in ['target', 'datetime', 'forecast_date', 'row_id']]

# # Split temporal 80/20
# split_idx = int(len(sample_df) * 0.8)
# train_data = sample_df.iloc[:split_idx]
# test_data = sample_df.iloc[split_idx:]

# X_train = train_data[feature_cols].dropna()
# y_train = train_data.loc[X_train.index, 'target']
# X_test = test_data[feature_cols].dropna()
# y_test = test_data.loc[X_test.index, 'target']

# print(f"Features totales: {len(feature_cols)}")
# print(f"Train: {len(X_train):,} | Test: {len(X_test):,}")

# # LightGBM
# model = lgb.LGBMRegressor(
#     n_estimators=100,
#     max_depth=6,
#     learning_rate=0.1,
#     random_state=42,
#     n_jobs=-1,
#     verbose=-1
# )

# model.fit(X_train, y_train)

# # Resultados
# pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, pred)

# print(f"MAE con is_daylight (3 meses): {mae:.4f}")
# print(f"Baseline original era: 89.3")

# # #Dataset original: 32,963,024
# # Últimos 3 meses: 4,674,024
# # Porcentaje: 14.2%
# # Daylight distribution:
# # is_daylight
# # 1    2592660
# # 0    2081364
# # Name: count, dtype: int64
# # Features totales: 44
# # Train: 3,739,219 | Test: 934,805
# # MAE con is_daylight (3 meses): 86.6747
# # Baseline original era: 89.3

In [19]:
#5to modelo lgb igual que el anterior , solo agregue binaria Weekend y no vi cambios
#Mae = 86.6747
# Preparar datos y tomar últimos 3 meses
# train5 = train5.sort_values(['prediction_unit_id', 'datetime'])
# train5['datetime'] = pd.to_datetime(train5['datetime'])

# # Últimos 3 meses
# max_date = train5['datetime'].max()
# cutoff_date = max_date - pd.DateOffset(months=3)
# sample_df = train5[train5['datetime'] >= cutoff_date].copy()

# print(f"Dataset original: {len(train5):,}")
# print(f"Últimos 3 meses: {len(sample_df):,}")
# print(f"Porcentaje: {len(sample_df)/len(train5)*100:.1f}%")

# # Features temporales
# sample_df['day_of_week'] = sample_df['datetime'].dt.dayofweek
# sample_df['month'] = sample_df['datetime'].dt.month

# # Crear is_daylight (1 si hay radiación solar, 0 si es noche)
# sample_df['is_daylight'] = (sample_df['direct_solar_radiation'] > 0).astype(int)

# # Crear weekend (1 para sábado/domingo, 0 para lunes-viernes)
# sample_df['weekend'] = (sample_df['day_of_week'] >= 5).astype(int)

# print(f"Daylight distribution:")
# print(sample_df['is_daylight'].value_counts())
# print(f"Weekend distribution:")
# print(sample_df['weekend'].value_counts())

# # Features (todas menos datetime/forecast_date/target/row_id)
# feature_cols = [col for col in sample_df.columns 
#                 if col not in ['target', 'datetime', 'forecast_date', 'row_id']]

# # Split temporal 80/20
# split_idx = int(len(sample_df) * 0.8)
# train_data = sample_df.iloc[:split_idx]
# test_data = sample_df.iloc[split_idx:]

# X_train = train_data[feature_cols].dropna()
# y_train = train_data.loc[X_train.index, 'target']
# X_test = test_data[feature_cols].dropna()
# y_test = test_data.loc[X_test.index, 'target']

# print(f"Features totales: {len(feature_cols)}")
# print(f"Train: {len(X_train):,} | Test: {len(X_test):,}")

# # LightGBM
# model = lgb.LGBMRegressor(
#     n_estimators=100,
#     max_depth=6,
#     learning_rate=0.1,
#     random_state=42,
#     n_jobs=-1,
#     verbose=-1
# )

# model.fit(X_train, y_train)

# # Resultados
# pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, pred)

# print(f"MAE con is_daylight + weekend (3 meses): {mae:.4f}")
# print(f"MAE anterior era: 86.67")

# # Dataset original: 32,963,024
# # Últimos 3 meses: 4,674,024
# # Porcentaje: 14.2%
# # Daylight distribution:
# # is_daylight
# # 1    2592660
# # 0    2081364
# # Name: count, dtype: int64
# # Weekend distribution:
# # weekend
# # 0    3341592
# # 1    1332432
# # Name: count, dtype: int64
# # Features totales: 45
# # Train: 3,739,219 | Test: 934,805
# # MAE con is_daylight + weekend (3 meses): 86.6747
# # MAE anterior era: 86.67

In [20]:
#6to modelo lgb , agregue variable binaria is_active_hours , horario de 8 a 17 hs binaria
#Mae = 90.4776 , empeoro el modelo

# # Usar train5 con las features ya agregadas
# train5 = train5.sort_values(['prediction_unit_id', 'datetime'])

# # Agregar is_active_hours (horario laboral/escolar lunes-viernes 8-17h)
# train5['is_active_hours'] = (
#     (train5['day_of_week'] < 5) &  # Lunes-viernes
#     (train5['hour'] >= 8) & 
#     (train5['hour'] <= 17)
# ).astype(int)

# # Últimos 3 meses para evitar problemas de memoria
# max_date = train5['datetime'].max()
# cutoff_date = max_date - pd.DateOffset(months=3)
# sample_df = train5[train5['datetime'] >= cutoff_date].copy()

# # Features (todas menos datetime/forecast_date/target/row_id)
# feature_cols = [col for col in sample_df.columns 
#                 if col not in ['target', 'datetime', 'forecast_date', 'row_id']]

# # Split temporal 80/20
# split_idx = int(len(sample_df) * 0.8)
# train_data = sample_df.iloc[:split_idx]
# test_data = sample_df.iloc[split_idx:]

# X_train = train_data[feature_cols].dropna()
# y_train = train_data.loc[X_train.index, 'target']
# X_test = test_data[feature_cols].dropna()
# y_test = test_data.loc[X_test.index, 'target']



# # LightGBM
# model = lgb.LGBMRegressor(
#     n_estimators=100,
#     max_depth=6,
#     learning_rate=0.1,
#     random_state=42,
#     n_jobs=-1,
#     verbose=-1
# )

# model.fit(X_train, y_train)

# # Resultados
# pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, pred)

# print(f"MAE con is_active_hours (3 meses): {mae:.4f}")
# print(f"MAE anterior (is_daylight): 86.67")

# MAE con is_active_hours (3 meses): 90.4776
# MAE anterior (is_daylight): 86.67

In [21]:
#7mo modelo lgb, agregue variable binaria is_sleeping_hours 22h-6h
#Mae = 98.3152 empeora el modelo , asi que no la saco

# train5 = train5.sort_values(['prediction_unit_id', 'datetime'])

# # Agregar is_sleeping_hours (22h-6h)
# train5['is_sleeping_hours'] = (
#     (train5['hour'] >= 22) | (train5['hour'] <= 6)
# ).astype(int)

# # Últimos 3 meses para evitar problemas de memoria
# max_date = train5['datetime'].max()
# cutoff_date = max_date - pd.DateOffset(months=3)
# sample_df = train5[train5['datetime'] >= cutoff_date].copy()

# print(f"Dataset original: {len(train5):,}")
# print(f"Últimos 3 meses: {len(sample_df):,}")
# print(f"Porcentaje: {len(sample_df)/len(train5)*100:.1f}%")

# print(f"Sleeping hours distribution:")
# print(sample_df['is_sleeping_hours'].value_counts())

# # Features (todas menos datetime/forecast_date/target/row_id)
# feature_cols = [col for col in sample_df.columns 
#                 if col not in ['target', 'datetime', 'forecast_date', 'row_id']]

# # Split temporal 80/20
# split_idx = int(len(sample_df) * 0.8)
# train_data = sample_df.iloc[:split_idx]
# test_data = sample_df.iloc[split_idx:]

# X_train = train_data[feature_cols].dropna()
# y_train = train_data.loc[X_train.index, 'target']
# X_test = test_data[feature_cols].dropna()
# y_test = test_data.loc[X_test.index, 'target']

# print(f"Features totales: {len(feature_cols)}")
# print(f"Train: {len(X_train):,} | Test: {len(X_test):,}")

# # LightGBM
# model = lgb.LGBMRegressor(
#     n_estimators=100,
#     max_depth=6,
#     learning_rate=0.1,
#     random_state=42,
#     n_jobs=-1,
#     verbose=-1
# )

# model.fit(X_train, y_train)

# # Resultados
# pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, pred)

# print(f"MAE con is_active_hours (3 meses): {mae:.4f}")
# print(f"MAE anterior (is_daylight): 86.67")

# Dataset original: 32,963,024
# Últimos 3 meses: 4,674,024
# Porcentaje: 14.2%
# Sleeping hours distribution:
# is_sleeping_hours
# 0    2923632
# 1    1750392
# Name: count, dtype: int64
# Features totales: 46
# Train: 3,739,219 | Test: 934,805
# MAE con is_active_hours (3 meses): 98.3152
# MAE anterior (is_daylight): 86.67

In [22]:
#8vo modelo lgb agregue 3 variables energeticas capacity_per_obs, production_obs_ratio, consumption_obs_ratio
#Mae = 76.2703 mejor bastante el modelo
# Usar train5 con features básicas
# train5 = train5.sort_values(['prediction_unit_id', 'datetime'])

# # Últimos 3 meses
# max_date = train5['datetime'].max()
# cutoff_date = max_date - pd.DateOffset(months=3)
# sample_df = train5[train5['datetime'] >= cutoff_date].copy()

# # Features energéticas 
# print("🔄 Calculando features energéticas...")

# energy_stats = []
# for unit_id in sample_df['prediction_unit_id'].unique():
#     unit_data = sample_df[sample_df['prediction_unit_id'] == unit_id]
    
#     # Usar installed_capacity como proxy de capacidad productiva
#     capacity = unit_data['installed_capacity'].iloc[0]
    
#     # Ratio de tipos: cuántas observaciones son production vs consumption
#     total_obs = len(unit_data)
#     production_obs = len(unit_data[unit_data['is_consumption'] == 0])
#     consumption_obs = len(unit_data[unit_data['is_consumption'] == 1])
    
#     production_ratio = production_obs / total_obs if total_obs > 0 else 0
#     consumption_ratio = consumption_obs / total_obs if total_obs > 0 else 0
    
#     energy_stats.append({
#         'prediction_unit_id': unit_id,
#         'capacity_per_obs': capacity / total_obs if total_obs > 0 else 0,
#         'production_obs_ratio': production_ratio,
#         'consumption_obs_ratio': consumption_ratio
#     })

# energy_df = pd.DataFrame(energy_stats)

# # Merge con datos principales
# sample_df = sample_df.merge(energy_df, on='prediction_unit_id', how='left')

# print(f"Últimos 3 meses: {len(sample_df):,}")
# print(f"Nuevas features: capacity_per_obs, production_obs_ratio, consumption_obs_ratio")

# # Features (todas menos datetime/forecast_date/target/row_id)
# feature_cols = [col for col in sample_df.columns 
#                 if col not in ['target', 'datetime', 'forecast_date', 'row_id']]

# # Split temporal 80/20
# split_idx = int(len(sample_df) * 0.8)
# train_data = sample_df.iloc[:split_idx]
# test_data = sample_df.iloc[split_idx:]

# X_train = train_data[feature_cols].dropna()
# y_train = train_data.loc[X_train.index, 'target']
# X_test = test_data[feature_cols].dropna()
# y_test = test_data.loc[X_test.index, 'target']

# print(f"Features totales: {len(feature_cols)}")
# print(f"Train: {len(X_train):,} | Test: {len(X_test):,}")

# # LightGBM
# model = lgb.LGBMRegressor(
#     n_estimators=100,
#     max_depth=6,
#     learning_rate=0.1,
#     random_state=42,
#     n_jobs=-1,
#     verbose=-1
# )

# model.fit(X_train, y_train)

# # Resultados
# pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, pred)

# print(f"MAE con features energéticas: {mae:.4f}")
# print(f"MAE anterior: 86.67")

# # BORRAR MEMORIA
# del model, X_train, X_test, y_train, y_test, pred, sample_df, train_data, test_data, energy_df, energy_stats
# import gc
# gc.collect()

# Últimos 3 meses: 4,674,024
# Nuevas features: capacity_per_obs, production_obs_ratio, consumption_obs_ratio
# Features totales: 48
# Train: 3,739,219 | Test: 934,805
# MAE con features energéticas: 76.2703
# MAE anterior: 86.67

In [23]:
#v9 no optimizado para poder usarlo #

# # Usar train5 con features básicas
# train5 = train5.sort_values(['prediction_unit_id', 'datetime'])

# # USAR 4 MESES para calcular ratio, luego reducir a 3
# max_date = train5['datetime'].max()
# cutoff_4m = max_date - pd.DateOffset(months=4)
# cutoff_3m = max_date - pd.DateOffset(months=3)

# sample_4m = train5[train5['datetime'] >= cutoff_4m].copy()
# print("🔄 Calculando ratio production/consumption histórico (30d)...")

# # MÉTODO SIMPLE Y DIRECTO - sin reindexing problemático
# sample_4m['prod_cons_ratio_30d'] = 0.0

# for unit_id in sample_4m['prediction_unit_id'].unique():
#     unit_data = sample_4m[sample_4m['prediction_unit_id'] == unit_id].copy()
#     unit_data = unit_data.sort_values('datetime')
    
#     # Para cada fila, calcular ratio con ventana de 30 días hacia atrás (SIMPLE)
#     for i in range(len(unit_data)):
#         current_time = unit_data.iloc[i]['datetime']
#         start_time = current_time - pd.Timedelta(days=30)
        
#         # Datos en ventana de 30 días
#         window_data = unit_data[
#             (unit_data['datetime'] >= start_time) & 
#             (unit_data['datetime'] < current_time)
#         ]
        
#         if len(window_data) > 0:
#             # Promedios simples
#             prod_mean = window_data[window_data['is_consumption'] == 0]['target'].mean()
#             cons_mean = window_data[window_data['is_consumption'] == 1]['target'].mean()
            
#             # Manejar NaN y división por cero
#             if pd.isna(prod_mean): prod_mean = 0
#             if pd.isna(cons_mean): cons_mean = 1
            
#             ratio = prod_mean / cons_mean if cons_mean != 0 else 0
#             sample_4m.iloc[i, sample_4m.columns.get_loc('prod_cons_ratio_30d')] = ratio

# # Reducir a 3 meses finales
# sample_df = sample_4m[sample_4m['datetime'] >= cutoff_3m].copy()
# print(f"Datos finales (3 meses): {len(sample_df):,}")

# # Features energéticas
# energy_stats = []
# for unit_id in sample_df['prediction_unit_id'].unique():
#     unit_data = sample_df[sample_df['prediction_unit_id'] == unit_id]
#     capacity = unit_data['installed_capacity'].iloc[0]
#     total_obs = len(unit_data)
#     production_obs = len(unit_data[unit_data['is_consumption'] == 0])
    
#     energy_stats.append({
#         'prediction_unit_id': unit_id,
#         'capacity_per_obs': capacity / total_obs if total_obs > 0 else 0,
#         'production_obs_ratio': production_obs / total_obs if total_obs > 0 else 0
#     })

# energy_df = pd.DataFrame(energy_stats)
# sample_df = sample_df.merge(energy_df, on='prediction_unit_id', how='left')

# # Split y entrenamiento
# feature_cols = [col for col in sample_df.columns 
#                 if col not in ['target', 'datetime', 'forecast_date', 'row_id']]

# split_idx = int(len(sample_df) * 0.8)
# X_train = sample_df.iloc[:split_idx][feature_cols].dropna()
# y_train = sample_df.iloc[:split_idx].loc[X_train.index, 'target']
# X_test = sample_df.iloc[split_idx:][feature_cols].dropna()
# y_test = sample_df.iloc[split_idx:].loc[X_test.index, 'target']

# print(f"Features: {len(feature_cols)} | Train: {len(X_train):,} | Test: {len(X_test):,}")

# model = lgb.LGBMRegressor(n_estimators=100, max_depth=6, learning_rate=0.1, 
#                          random_state=42, n_jobs=-1, verbose=-1)
# model.fit(X_train, y_train)

# pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, pred)
# print(f"✅ MAE con ratio 30d: {mae:.4f}")
# print(f"📊 MAE anterior: 76.27")

# print(f"📈 Ratio stats: min={sample_df['prod_cons_ratio_30d'].min():.3f}, max={sample_df['prod_cons_ratio_30d'].max():.3f}, NaNs={sample_df['prod_cons_ratio_30d'].isna().sum()}")

# # Limpiar memoria
# del model, X_train, X_test, y_train, y_test, pred, sample_df, sample_4m, energy_df, energy_stats
# import gc
# gc.collect()

In [24]:
# #10mo modelo lgb mantengo las 3 anteriores y pruebo  Capacidad instalada per capita (installed_capacity / eic_count)
# # Usar train5 con features básicas
# train5 = train5.sort_values(['prediction_unit_id', 'datetime'])

# # Últimos 3 meses
# max_date = train5['datetime'].max()
# cutoff_date = max_date - pd.DateOffset(months=3)
# sample_df = train5[train5['datetime'] >= cutoff_date].copy()

# print("🔄 Calculando features energéticas + capacity per capita...")

# # Features energéticas existentes + NUEVA: capacity per capita
# energy_stats = []
# for unit_id in sample_df['prediction_unit_id'].unique():
#     unit_data = sample_df[sample_df['prediction_unit_id'] == unit_id]
    
#     capacity = unit_data['installed_capacity'].iloc[0]
#     eic_count = unit_data['eic_count'].iloc[0]
#     total_obs = len(unit_data)
#     production_obs = len(unit_data[unit_data['is_consumption'] == 0])
#     consumption_obs = len(unit_data[unit_data['is_consumption'] == 1])
    
#     energy_stats.append({
#         'prediction_unit_id': unit_id,
#         'capacity_per_obs': capacity / total_obs if total_obs > 0 else 0,
#         'production_obs_ratio': production_obs / total_obs if total_obs > 0 else 0,
#         'consumption_obs_ratio': consumption_obs / total_obs if total_obs > 0 else 0,
#         'capacity_per_capita': capacity / eic_count if eic_count > 0 else 0  # NUEVA FEATURE
#     })

# energy_df = pd.DataFrame(energy_stats)
# sample_df = sample_df.merge(energy_df, on='prediction_unit_id', how='left')

# print(f"Datos con features energéticas: {len(sample_df):,}")

# # Features (todas menos datetime/forecast_date/target/row_id)
# feature_cols = [col for col in sample_df.columns 
#                 if col not in ['target', 'datetime', 'forecast_date', 'row_id']]

# # Split temporal 80/20
# split_idx = int(len(sample_df) * 0.8)
# train_data = sample_df.iloc[:split_idx]
# test_data = sample_df.iloc[split_idx:]

# X_train = train_data[feature_cols].dropna()
# y_train = train_data.loc[X_train.index, 'target']
# X_test = test_data[feature_cols].dropna()
# y_test = test_data.loc[X_test.index, 'target']

# print(f"Features totales: {len(feature_cols)}")
# print(f"Train: {len(X_train):,} | Test: {len(X_test):,}")

# # LightGBM
# model = lgb.LGBMRegressor(
#     n_estimators=100,
#     max_depth=6,
#     learning_rate=0.1,
#     random_state=42,
#     n_jobs=-1,
#     verbose=-1
# )

# model.fit(X_train, y_train)

# # Resultados
# pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, pred)
# print(f"✅ MAE con capacity per capita: {mae:.4f}")
# print(f"📊 MAE anterior: 76.27")

# # LIMPIAR MEMORIA
# del model, X_train, X_test, y_train, y_test, pred, sample_df, train_data, test_data
# del energy_df, energy_stats, feature_importance
# import gc
# gc.collect()
# print("🧹 Memoria liberada")

In [25]:
train5.head()

,row_id,target,datetime,forecast_date,hour,data_block_id,prediction_unit_id,is_business,product_type,county,latitude,longitude,lowest_price_per_mwh,highest_price_per_mwh,euros_per_mwh,eic_count,installed_capacity,ftemperature,temperature,fdewpoint,dewpoint,fcloudcover_high,cloudcover_high,fcloudcover_low,cloudcover_low,fcloudcover_mid,cloudcover_mid,fcloudcover_total,cloudcover_total,f10_metre_u_wind_component,windspeed_10m,f10_metre_v_wind_component,winddirection_10m,fdirect_solar_radiation,direct_solar_radiation,fsurface_solar_radiation_downwards,shortwave_radiation,diffuse_radiation,fsnowfall,snowfall,ftotal_precipitation,rain,surface_pressure,weather_forecast_hour,is_consumption,day_of_week,month,is_daylight,weekend
6,366,0.496,2021-09-01 03:00:00,2021-09-01 03:00:00,3,0,0,0,1,0,59.099998,24.200001,45.23,46.32,86.879997,108.0,952.890015,12.681543,12.4,9.783228,9.7,0.023590,11.0,0.002380,10.0,0.001251,0.0,0.026398,12.0,1.840991,4.222222,-3.857846,338.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1009.200012,1.0,0,2,9,0,0
7,366,0.496,2021-09-01 03:00:00,2021-09-01 03:00:00,3,0,0,0,1,0,59.099998,25.200001,45.23,46.32,86.879997,108.0,952.890015,12.868921,12.3,9.498316,9.6,0.431854,6.0,0.211182,23.0,0.006790,1.0,0.548508,23.0,1.505298,4.027778,-3.590024,337.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1004.200012,1.0,0,2,9,0,0
8,366,0.496,2021-09-01 03:00:00,2021-09-01 03:00:00,3,0,0,0,1,0,59.400002,22.700001,45.23,46.32,86.879997,108.0,952.890015,15.041773,15.2,11.860376,11.8,0.134674,18.0,0.202515,7.0,0.003906,0.0,0.308930,12.0,3.185351,9.055555,-8.173276,338.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1015.000000,1.0,0,2,9,0,0
9,366,0.496,2021-09-01 03:00:00,2021-09-01 03:00:00,3,0,0,0,1,0,59.400002,23.200001,45.23,46.32,86.879997,108.0,952.890015,14.632105,14.9,11.773584,11.5,0.255188,19.0,0.036774,6.0,0.026245,0.0,0.286194,11.0,3.474780,8.361111,-8.008969,335.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1014.500000,1.0,0,2,9,0,0
10,366,0.496,2021-09-01 03:00:00,2021-09-01 03:00:00,3,0,0,0,1,0,59.400002,23.700001,45.23,46.32,86.879997,108.0,952.890015,14.480005,12.8,11.581568,10.4,0.338074,12.0,0.037109,4.0,0.016510,1.0,0.368134,8.0,3.211841,5.416667,-7.426206,337.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1014.000000,1.0,0,2,9,0,0


In [26]:
# train5["fsurface_solar_radiation_downwards"].describe()

# count    3.296302e+07
# mean     1.103279e+02
# std      1.714488e+02
# min     -3.258333e-01
# 25%      0.000000e+00
# 50%      5.706424e-01
# 75%      1.429561e+02
# max      8.487144e+02
# Name: fsurface_solar_radiation_downwards, dtype: float64

train5["fsurface_solar_radiation_downwards"].value_counts()

fsurface_solar_radiation_downwards
0.000000      15297128
0.284444         39564
0.142222         35932
0.568889         28742
0.071111         21062
                ...   
169.200272           2
320.213043           2
48.876667            2
42.334446            2
268.698120           2
Name: count, Length: 1291576, dtype: int64

In [27]:
# #11vo modelo lgb 4 feature electrica y pruebo panel_efficiency
# # ✅ MAE con panel efficiency: 85.0309
# # Usar train5 con features básicas
# train5 = train5.sort_values(['prediction_unit_id', 'datetime'])

# # Últimos 3 meses
# max_date = train5['datetime'].max()
# cutoff_date = max_date - pd.DateOffset(months=3)
# sample_df = train5[train5['datetime'] >= cutoff_date].copy()

# print("🔄 Calculando features energéticas + capacity per capita + panel efficiency...")

# # Features energéticas existentes + NUEVAS: capacity per capita + panel efficiency
# energy_stats = []
# for unit_id in sample_df['prediction_unit_id'].unique():
#     unit_data = sample_df[sample_df['prediction_unit_id'] == unit_id]
    
#     capacity = unit_data['installed_capacity'].iloc[0]
#     eic_count = unit_data['eic_count'].iloc[0]
#     total_obs = len(unit_data)
#     production_obs = len(unit_data[unit_data['is_consumption'] == 0])
#     consumption_obs = len(unit_data[unit_data['is_consumption'] == 1])
    
#     # Calcular eficiencia de paneles (production vs solar_radiation cuando is_consumption=False)
#     production_data = unit_data[unit_data['is_consumption'] == 0]
#     if len(production_data) > 0 and 'fsurface_solar_radiation_downwards' in production_data.columns:
#         solar_rad = production_data['fsurface_solar_radiation_downwards']
#         production = production_data['target']
#         # Filtrar valores válidos (solar > 0 para evitar división por 0)
#         valid_mask = (solar_rad > 0) & (production >= 0)
#         if valid_mask.sum() > 0:
#             panel_efficiency = (production[valid_mask] / solar_rad[valid_mask]).mean()
#         else:
#             panel_efficiency = 0
#     else:
#         panel_efficiency = 0
    
#     energy_stats.append({
#         'prediction_unit_id': unit_id,
#         'capacity_per_obs': capacity / total_obs if total_obs > 0 else 0,
#         'production_obs_ratio': production_obs / total_obs if total_obs > 0 else 0,
#         'consumption_obs_ratio': consumption_obs / total_obs if total_obs > 0 else 0,
#         'capacity_per_capita': capacity / eic_count if eic_count > 0 else 0,
#         'panel_efficiency': panel_efficiency  # NUEVA FEATURE
#     })

# energy_df = pd.DataFrame(energy_stats)
# sample_df = sample_df.merge(energy_df, on='prediction_unit_id', how='left')

# print(f"Datos con features energéticas: {len(sample_df):,}")

# # Features (todas menos datetime/forecast_date/target/row_id)
# feature_cols = [col for col in sample_df.columns 
#                 if col not in ['target', 'datetime', 'forecast_date', 'row_id']]

# # Split temporal 80/20
# split_idx = int(len(sample_df) * 0.8)
# train_data = sample_df.iloc[:split_idx]
# test_data = sample_df.iloc[split_idx:]

# X_train = train_data[feature_cols].dropna()
# y_train = train_data.loc[X_train.index, 'target']
# X_test = test_data[feature_cols].dropna()
# y_test = test_data.loc[X_test.index, 'target']

# print(f"Features totales: {len(feature_cols)}")
# print(f"Train: {len(X_train):,} | Test: {len(X_test):,}")

# # LightGBM
# model = lgb.LGBMRegressor(
#     n_estimators=100,
#     max_depth=6,
#     learning_rate=0.1,
#     random_state=42,
#     n_jobs=-1,
#     verbose=-1
# )

# model.fit(X_train, y_train)

# # Resultados
# pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, pred)
# print(f"✅ MAE con panel efficiency: {mae:.4f}")
# print(f"📊 MAE anterior: 72.3572")

# # Verificar nuevas features
# print(f"📈 Panel efficiency stats:")
# print(f"   - Min: {sample_df['panel_efficiency'].min():.4f}")
# print(f"   - Max: {sample_df['panel_efficiency'].max():.4f}")
# print(f"   - Mean: {sample_df['panel_efficiency'].mean():.4f}")
# print(f"   - NaN count: {sample_df['panel_efficiency'].isna().sum()}")

# # LIMPIAR MEMORIA
# del model, X_train, X_test, y_train, y_test, pred, sample_df, train_data, test_data
# del energy_df, energy_stats
# import gc
# gc.collect()
# print("🧹 Memoria liberada")

In [28]:
# #12vo modelo lgb , lo anterior pero ahora efficiency panel es log
# # ✅ MAE con panel efficiency LOG: 85.3200
# # 📊 MAE anterior: 72.3572

# # Usar train5 con features básicas
# train5 = train5.sort_values(['prediction_unit_id', 'datetime'])

# # Últimos 3 meses
# max_date = train5['datetime'].max()
# cutoff_date = max_date - pd.DateOffset(months=3)
# sample_df = train5[train5['datetime'] >= cutoff_date].copy()

# print("🔄 Calculando features energéticas + capacity per capita + panel efficiency LOG...")

# # Features energéticas existentes + NUEVAS: capacity per capita + panel efficiency LOG
# energy_stats = []
# for unit_id in sample_df['prediction_unit_id'].unique():
#     unit_data = sample_df[sample_df['prediction_unit_id'] == unit_id]
    
#     capacity = unit_data['installed_capacity'].iloc[0]
#     eic_count = unit_data['eic_count'].iloc[0]
#     total_obs = len(unit_data)
#     production_obs = len(unit_data[unit_data['is_consumption'] == 0])
#     consumption_obs = len(unit_data[unit_data['is_consumption'] == 1])
    
#     # Calcular eficiencia de paneles LOG (production vs solar_radiation cuando is_consumption=False)
#     production_data = unit_data[unit_data['is_consumption'] == 0]
#     if len(production_data) > 0 and 'fsurface_solar_radiation_downwards' in production_data.columns:
#         solar_rad = production_data['fsurface_solar_radiation_downwards']
#         production = production_data['target']
#         # Filtrar valores válidos (solar > 0 para evitar división por 0)
#         valid_mask = (solar_rad > 0) & (production >= 0)
#         if valid_mask.sum() > 0:
#             efficiency_ratio = (production[valid_mask] / solar_rad[valid_mask]).mean()
#             # Aplicar log pero manejando valores <= 1
#             if efficiency_ratio > 1:
#                 panel_efficiency_log = np.log(efficiency_ratio)
#             elif efficiency_ratio > 0:
#                 panel_efficiency_log = -np.log(1/efficiency_ratio)  # log inverso para valores < 1
#             else:
#                 panel_efficiency_log = 0
#         else:
#             panel_efficiency_log = 0
#     else:
#         panel_efficiency_log = 0
    
#     energy_stats.append({
#         'prediction_unit_id': unit_id,
#         'capacity_per_obs': capacity / total_obs if total_obs > 0 else 0,
#         'production_obs_ratio': production_obs / total_obs if total_obs > 0 else 0,
#         'consumption_obs_ratio': consumption_obs / total_obs if total_obs > 0 else 0,
#         'capacity_per_capita': capacity / eic_count if eic_count > 0 else 0,
#         'panel_efficiency_log': panel_efficiency_log  # NUEVA FEATURE CON LOG
#     })

# energy_df = pd.DataFrame(energy_stats)
# sample_df = sample_df.merge(energy_df, on='prediction_unit_id', how='left')

# print(f"Datos con features energéticas: {len(sample_df):,}")

# # Features (todas menos datetime/forecast_date/target/row_id)
# feature_cols = [col for col in sample_df.columns 
#                 if col not in ['target', 'datetime', 'forecast_date', 'row_id']]

# # Split temporal 80/20
# split_idx = int(len(sample_df) * 0.8)
# train_data = sample_df.iloc[:split_idx]
# test_data = sample_df.iloc[split_idx:]

# X_train = train_data[feature_cols].dropna()
# y_train = train_data.loc[X_train.index, 'target']
# X_test = test_data[feature_cols].dropna()
# y_test = test_data.loc[X_test.index, 'target']

# print(f"Features totales: {len(feature_cols)}")
# print(f"Train: {len(X_train):,} | Test: {len(X_test):,}")

# # LightGBM
# model = lgb.LGBMRegressor(
#     n_estimators=100,
#     max_depth=6,
#     learning_rate=0.1,
#     random_state=42,
#     n_jobs=-1,
#     verbose=-1
# )

# model.fit(X_train, y_train)

# # Resultados
# pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, pred)
# print(f"✅ MAE con panel efficiency LOG: {mae:.4f}")
# print(f"📊 MAE anterior: 72.3572")

# # Verificar nuevas features
# print(f"📈 Capacity per capita stats:")
# print(f"   - Min: {sample_df['capacity_per_capita'].min():.4f}")
# print(f"   - Max: {sample_df['capacity_per_capita'].max():.4f}")
# print(f"   - Mean: {sample_df['capacity_per_capita'].mean():.4f}")
# print(f"   - NaN count: {sample_df['capacity_per_capita'].isna().sum()}")

# print(f"📈 Panel efficiency LOG stats:")
# print(f"   - Min: {sample_df['panel_efficiency_log'].min():.4f}")
# print(f"   - Max: {sample_df['panel_efficiency_log'].max():.4f}")
# print(f"   - Mean: {sample_df['panel_efficiency_log'].mean():.4f}")
# print(f"   - NaN count: {sample_df['panel_efficiency_log'].isna().sum()}")

# # LIMPIAR MEMORIA
# del model, X_train, X_test, y_train, y_test, pred, sample_df, train_data, test_data
# del energy_df, energy_stats
# import gc
# gc.collect()
# print("🧹 Memoria liberada")

In [29]:
# #13vo modelo probando volatilidad de precio de electricidad 7 dias
# # ✅ MAE con electricity price volatility: 73.2246
# # 📊 MAE anterior: 72.3572
# # Usar train5 con features básicas
# train5 = train5.sort_values(['prediction_unit_id', 'datetime'])

# # USAR 120 días para calcular volatilidad, luego reducir a 90
# max_date = train5['datetime'].max()
# cutoff_120d = max_date - pd.DateOffset(days=120)  # 120 días para calcular volatilidad
# cutoff_90d = max_date - pd.DateOffset(days=90)    # 90 días finales

# # Datos de 120 días para calcular volatilidad
# sample_120d = train5[train5['datetime'] >= cutoff_120d].copy()

# print("🔄 Calculando volatilidad de precios (7d rolling) con 120 días...")

# # Calcular volatilidad de electricity prices (rolling std 7 días)
# sample_120d['electricity_price_volatility_7d'] = 0.0

# for unit_id in sample_120d['prediction_unit_id'].unique():
#     unit_data = sample_120d[sample_120d['prediction_unit_id'] == unit_id].copy()
#     unit_data = unit_data.sort_values('datetime')
    
#     if 'euros_per_mwh' in unit_data.columns and len(unit_data) > 0:
#         # Rolling std de 7 días para electricity prices
#         unit_data = unit_data.set_index('datetime')
#         price_vol = unit_data['euros_per_mwh'].rolling('7D', min_periods=1).std()
        
#         # Manejar NaN y resetear index
#         price_vol = price_vol.fillna(0)
#         unit_data = unit_data.reset_index()
        
#         # Asignar volatilidad al dataframe principal
#         mask = sample_120d['prediction_unit_id'] == unit_id
#         sample_120d.loc[mask, 'electricity_price_volatility_7d'] = price_vol.values

# # AHORA reducir a últimos 90 días (con volatilidad ya calculada)
# sample_df = sample_120d[sample_120d['datetime'] >= cutoff_90d].copy()
# print(f"Datos finales (90 días con volatilidad): {len(sample_df):,}")

# print("🔄 Calculando features energéticas + capacity per capita...")

# # Features energéticas existentes + capacity per capita (SIN panel_efficiency)
# energy_stats = []
# for unit_id in sample_df['prediction_unit_id'].unique():
#     unit_data = sample_df[sample_df['prediction_unit_id'] == unit_id]
    
#     capacity = unit_data['installed_capacity'].iloc[0]
#     eic_count = unit_data['eic_count'].iloc[0]
#     total_obs = len(unit_data)
#     production_obs = len(unit_data[unit_data['is_consumption'] == 0])
#     consumption_obs = len(unit_data[unit_data['is_consumption'] == 1])
    
#     energy_stats.append({
#         'prediction_unit_id': unit_id,
#         'capacity_per_obs': capacity / total_obs if total_obs > 0 else 0,
#         'production_obs_ratio': production_obs / total_obs if total_obs > 0 else 0,
#         'consumption_obs_ratio': consumption_obs / total_obs if total_obs > 0 else 0,
#         'capacity_per_capita': capacity / eic_count if eic_count > 0 else 0
#     })

# energy_df = pd.DataFrame(energy_stats)
# sample_df = sample_df.merge(energy_df, on='prediction_unit_id', how='left')

# print(f"Datos con features energéticas + volatilidad: {len(sample_df):,}")

# # Features (todas menos datetime/forecast_date/target/row_id)
# feature_cols = [col for col in sample_df.columns 
#                 if col not in ['target', 'datetime', 'forecast_date', 'row_id']]

# # Split temporal 80/20
# split_idx = int(len(sample_df) * 0.8)
# train_data = sample_df.iloc[:split_idx]
# test_data = sample_df.iloc[split_idx:]

# X_train = train_data[feature_cols].dropna()
# y_train = train_data.loc[X_train.index, 'target']
# X_test = test_data[feature_cols].dropna()
# y_test = test_data.loc[X_test.index, 'target']

# print(f"Features totales: {len(feature_cols)}")
# print(f"Train: {len(X_train):,} | Test: {len(X_test):,}")

# # LightGBM
# model = lgb.LGBMRegressor(
#     n_estimators=100,
#     max_depth=6,
#     learning_rate=0.1,
#     random_state=42,
#     n_jobs=-1,
#     verbose=-1
# )

# model.fit(X_train, y_train)

# # Resultados
# pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, pred)
# print(f"✅ MAE con electricity price volatility: {mae:.4f}")
# print(f"📊 MAE anterior: 72.3572")

# # Verificar nuevas features
# print(f"📈 Electricity price volatility 7d stats:")
# print(f"   - Min: {sample_df['electricity_price_volatility_7d'].min():.4f}")
# print(f"   - Max: {sample_df['electricity_price_volatility_7d'].max():.4f}")
# print(f"   - Mean: {sample_df['electricity_price_volatility_7d'].mean():.4f}")
# print(f"   - NaN count: {sample_df['electricity_price_volatility_7d'].isna().sum()}")

# # LIMPIAR MEMORIA
# del model, X_train, X_test, y_train, y_test, pred, sample_df, sample_120d, train_data, test_data
# del energy_df, energy_stats
# import gc
# gc.collect()
# print("🧹 Memoria liberada")

In [30]:
# #14vo modelo ademas de electricity_volatility pruebo con gas
# # ✅ MAE con electricity + gas price volatility: 70.6517
# # 📊 MAE anterior: 73.2246

# # Usar train5 con features básicas
# train5 = train5.sort_values(['prediction_unit_id', 'datetime'])

# # USAR 120 días para calcular volatilidad, luego reducir a 90
# max_date = train5['datetime'].max()
# cutoff_120d = max_date - pd.DateOffset(days=120)  # 120 días para calcular volatilidad
# cutoff_90d = max_date - pd.DateOffset(days=90)    # 90 días finales

# # Datos de 120 días para calcular volatilidad
# sample_120d = train5[train5['datetime'] >= cutoff_120d].copy()

# print("🔄 Calculando volatilidad de electricity y gas prices (7d rolling) con 120 días...")

# # Calcular volatilidad de electricity y gas prices (rolling std 7 días)
# sample_120d['electricity_price_volatility_7d'] = 0.0
# sample_120d['gas_price_volatility_7d'] = 0.0

# for unit_id in sample_120d['prediction_unit_id'].unique():
#     unit_data = sample_120d[sample_120d['prediction_unit_id'] == unit_id].copy()
#     unit_data = unit_data.sort_values('datetime')
    
#     if len(unit_data) > 0:
#         unit_data_indexed = unit_data.set_index('datetime')
        
#         # Volatilidad electricity prices
#         if 'euros_per_mwh' in unit_data.columns:
#             elec_vol = unit_data_indexed['euros_per_mwh'].rolling('7D', min_periods=1).std().fillna(0)
#         else:
#             elec_vol = pd.Series(0, index=unit_data_indexed.index)
        
#         # Volatilidad gas prices
#         if 'lowest_price_per_mwh' in unit_data.columns:
#             gas_vol = unit_data_indexed['lowest_price_per_mwh'].rolling('7D', min_periods=1).std().fillna(0)
#         else:
#             gas_vol = pd.Series(0, index=unit_data_indexed.index)
        
#         # Asignar volatilidades al dataframe principal usando iloc
#         mask = sample_120d['prediction_unit_id'] == unit_id
#         indices = sample_120d[mask].index
        
#         sample_120d.loc[indices, 'electricity_price_volatility_7d'] = elec_vol.values
#         sample_120d.loc[indices, 'gas_price_volatility_7d'] = gas_vol.values

# # AHORA reducir a últimos 90 días (conservando TODAS las columnas)
# sample_df = sample_120d[sample_120d['datetime'] >= cutoff_90d].copy()
# print(f"Datos finales (90 días con volatilidades): {len(sample_df):,}")

# print("🔄 Calculando features energéticas + capacity per capita...")

# # Features energéticas existentes + capacity per capita
# energy_stats = []
# for unit_id in sample_df['prediction_unit_id'].unique():
#     unit_data = sample_df[sample_df['prediction_unit_id'] == unit_id]
    
#     capacity = unit_data['installed_capacity'].iloc[0]
#     eic_count = unit_data['eic_count'].iloc[0]
#     total_obs = len(unit_data)
#     production_obs = len(unit_data[unit_data['is_consumption'] == 0])
#     consumption_obs = len(unit_data[unit_data['is_consumption'] == 1])
    
#     energy_stats.append({
#         'prediction_unit_id': unit_id,
#         'capacity_per_obs': capacity / total_obs if total_obs > 0 else 0,
#         'production_obs_ratio': production_obs / total_obs if total_obs > 0 else 0,
#         'consumption_obs_ratio': consumption_obs / total_obs if total_obs > 0 else 0,
#         'capacity_per_capita': capacity / eic_count if eic_count > 0 else 0
#     })

# energy_df = pd.DataFrame(energy_stats)
# sample_df = sample_df.merge(energy_df, on='prediction_unit_id', how='left')

# print(f"Datos con features energéticas + volatilidades: {len(sample_df):,}")


# # Features (todas menos datetime/forecast_date/target/row_id)
# feature_cols = [col for col in sample_df.columns 
#                 if col not in ['target', 'datetime', 'forecast_date', 'row_id']]

# # Split temporal 80/20
# split_idx = int(len(sample_df) * 0.8)
# train_data = sample_df.iloc[:split_idx]
# test_data = sample_df.iloc[split_idx:]

# X_train = train_data[feature_cols].dropna()
# y_train = train_data.loc[X_train.index, 'target']
# X_test = test_data[feature_cols].dropna()
# y_test = test_data.loc[X_test.index, 'target']

# print(f"Features totales: {len(feature_cols)}")
# print(f"Train: {len(X_train):,} | Test: {len(X_test):,}")

# # LightGBM
# model = lgb.LGBMRegressor(
#     n_estimators=100,
#     max_depth=6,
#     learning_rate=0.1,
#     random_state=42,
#     n_jobs=-1,
#     verbose=-1
# )

# model.fit(X_train, y_train)

# # Resultados
# pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, pred)
# print(f"✅ MAE con electricity + gas price volatility: {mae:.4f}")
# print(f"📊 MAE anterior: 73.2246")

# # Verificar nuevas features

# if 'gas_price_volatility_7d' in sample_df.columns:
#     print(f"📈 Gas price volatility 7d stats:")
#     print(f"   - Min: {sample_df['gas_price_volatility_7d'].min():.4f}")
#     print(f"   - Max: {sample_df['gas_price_volatility_7d'].max():.4f}")
#     print(f"   - Mean: {sample_df['gas_price_volatility_7d'].mean():.4f}")
#     print(f"   - NaN count: {sample_df['gas_price_volatility_7d'].isna().sum()}")

# # LIMPIAR MEMORIA
# del model, X_train, X_test, y_train, y_test, pred, sample_df, sample_120d, train_data, test_data
# del energy_df, energy_stats
# import gc
# gc.collect()
# print("🧹 Memoria liberada")

In [31]:
# CREAR TRAIN6 con 150 días y todas las features consolidadas
print("🔄 Creando train6 con 150 días...")

# Usar train5 como base
train5 = train5.sort_values(['prediction_unit_id', 'datetime'])

# Últimos 150 días para train6
max_date = train5['datetime'].max()
cutoff_150d = max_date - pd.DateOffset(days=150)
train6 = train5[train5['datetime'] >= cutoff_150d].copy()

print(f"📊 train6 creado: {len(train6):,} filas ({len(train6['prediction_unit_id'].unique())} units)")

print("🔄 Calculando features energéticas...")

# Features energéticas
energy_stats = []
for unit_id in train6['prediction_unit_id'].unique():
    unit_data = train6[train6['prediction_unit_id'] == unit_id]
    
    capacity = unit_data['installed_capacity'].iloc[0]
    eic_count = unit_data['eic_count'].iloc[0]
    total_obs = len(unit_data)
    production_obs = len(unit_data[unit_data['is_consumption'] == 0])
    consumption_obs = len(unit_data[unit_data['is_consumption'] == 1])
    
    energy_stats.append({
        'prediction_unit_id': unit_id,
        'capacity_per_obs': capacity / total_obs if total_obs > 0 else 0,
        'production_obs_ratio': production_obs / total_obs if total_obs > 0 else 0,
        'consumption_obs_ratio': consumption_obs / total_obs if total_obs > 0 else 0,
        'capacity_per_capita': capacity / eic_count if eic_count > 0 else 0
    })

energy_df = pd.DataFrame(energy_stats)
train6 = train6.merge(energy_df, on='prediction_unit_id', how='left')

print("🔄 Calculando volatilidades de precios (7d rolling)...")

# Volatilidades de precios (7d rolling)
train6['electricity_price_volatility_7d'] = 0.0
train6['gas_price_volatility_7d'] = 0.0

for unit_id in train6['prediction_unit_id'].unique():
    unit_data = train6[train6['prediction_unit_id'] == unit_id].copy()
    unit_data = unit_data.sort_values('datetime')
    
    if len(unit_data) > 0:
        unit_data_indexed = unit_data.set_index('datetime')
        
        # Volatilidad electricity prices
        if 'euros_per_mwh' in unit_data.columns:
            elec_vol = unit_data_indexed['euros_per_mwh'].rolling('7D', min_periods=1).std().fillna(0)
        else:
            elec_vol = pd.Series(0, index=unit_data_indexed.index)
        
        # Volatilidad gas prices
        if 'lowest_price_per_mwh' in unit_data.columns:
            gas_vol = unit_data_indexed['lowest_price_per_mwh'].rolling('7D', min_periods=1).std().fillna(0)
        else:
            gas_vol = pd.Series(0, index=unit_data_indexed.index)
        
        # Asignar volatilidades
        mask = train6['prediction_unit_id'] == unit_id
        indices = train6[mask].index
        
        train6.loc[indices, 'electricity_price_volatility_7d'] = elec_vol.values
        train6.loc[indices, 'gas_price_volatility_7d'] = gas_vol.values

print("✅ train6 creado con todas las features:")
print(f"   📊 Datos: {len(train6):,} filas")
print(f"   📊 Período: {train6['datetime'].min()} a {train6['datetime'].max()}")
print(f"   🔧 Features energéticas: 4 (capacity_per_obs, production_obs_ratio, consumption_obs_ratio, capacity_per_capita)")
print(f"   📈 Volatilidades: 2 (electricity_price_volatility_7d, gas_price_volatility_7d)")

# Verificar features creadas
feature_check = [
    'capacity_per_obs', 'production_obs_ratio', 'consumption_obs_ratio', 
    'capacity_per_capita', 'electricity_price_volatility_7d', 'gas_price_volatility_7d'
]

print("\n🔍 Verificación de features:")
for feature in feature_check:
    if feature in train6.columns:
        print(f"   ✅ {feature}: OK")
    else:
        print(f"   ❌ {feature}: FALTA")

# Estadísticas básicas de las nuevas features
print(f"\n📈 Estadísticas rápidas:")
for feature in feature_check:
    if feature in train6.columns:
        print(f"   {feature}: min={train6[feature].min():.3f}, max={train6[feature].max():.3f}, mean={train6[feature].mean():.3f}")

print(f"\n✨ train6 listo para usar con {len(train6.columns)} columnas totales")

# Limpiar memoria temporal
del energy_df, energy_stats
import gc
gc.collect()
print("🧹 Memoria temporal liberada")

🔄 Creando train6 con 150 días...
📊 train6 creado: 7,821,024 filas (68 units)
🔄 Calculando features energéticas...
🔄 Calculando volatilidades de precios (7d rolling)...
✅ train6 creado con todas las features:
   📊 Datos: 7,821,024 filas
   📊 Período: 2022-12-31 10:00:00 a 2023-05-30 10:00:00
   🔧 Features energéticas: 4 (capacity_per_obs, production_obs_ratio, consumption_obs_ratio, capacity_per_capita)
   📈 Volatilidades: 2 (electricity_price_volatility_7d, gas_price_volatility_7d)

🔍 Verificación de features:
   ✅ capacity_per_obs: OK
   ✅ production_obs_ratio: OK
   ✅ consumption_obs_ratio: OK
   ✅ capacity_per_capita: OK
   ✅ electricity_price_volatility_7d: OK
   ✅ gas_price_volatility_7d: OK

📈 Estadísticas rápidas:
   capacity_per_obs: min=0.000, max=0.091, mean=0.014
   production_obs_ratio: min=0.500, max=0.500, mean=0.500
   consumption_obs_ratio: min=0.500, max=0.500, mean=0.500
   capacity_per_capita: min=0.438, max=213.750, mean=26.458
   electricity_price_volatility_7d: mi

In [32]:
train6.head()

,row_id,target,datetime,forecast_date,hour,data_block_id,prediction_unit_id,is_business,product_type,county,latitude,longitude,lowest_price_per_mwh,highest_price_per_mwh,euros_per_mwh,eic_count,installed_capacity,ftemperature,temperature,fdewpoint,dewpoint,fcloudcover_high,cloudcover_high,fcloudcover_low,cloudcover_low,fcloudcover_mid,cloudcover_mid,fcloudcover_total,cloudcover_total,f10_metre_u_wind_component,windspeed_10m,f10_metre_v_wind_component,winddirection_10m,fdirect_solar_radiation,direct_solar_radiation,fsurface_solar_radiation_downwards,shortwave_radiation,diffuse_radiation,fsnowfall,snowfall,ftotal_precipitation,rain,surface_pressure,weather_forecast_hour,is_consumption,day_of_week,month,is_daylight,weekend,capacity_per_obs,production_obs_ratio,consumption_obs_ratio,capacity_per_capita,electricity_price_volatility_7d,gas_price_volatility_7d
0,1535708,1.379,2022-12-31 10:00:00,2022-12-31 10:00:00,10,486,0,0,1,0,59.099998,24.200001,74.0,86.199997,3.29,417.0,4001.080078,2.458154,2.5,1.860742,1.8,0.989075,95.0,1.0,100.0,1.000000,97.0,0.999996,100.0,0.320052,8.777778,8.630277,187.0,-0.008889,0.0,5.534878,1.0,1.0,0.000023,0.0,0.000508,0.7,995.500000,33.0,0,5,12,0,1,0.016344,0.5,0.5,9.594916,0.0,0.0
1,1535708,1.379,2022-12-31 10:00:00,2022-12-31 10:00:00,10,486,0,0,1,0,59.099998,25.200001,74.0,86.199997,3.29,417.0,4001.080078,1.464502,2.2,0.857690,1.0,0.897797,98.0,1.0,40.0,1.000000,96.0,0.999996,100.0,0.415511,7.666667,7.309232,187.0,0.008889,0.0,15.983768,3.0,3.0,0.000063,0.0,0.000489,0.2,993.000000,33.0,0,5,12,0,1,0.016344,0.5,0.5,9.594916,0.0,0.0
2,1535708,1.379,2022-12-31 10:00:00,2022-12-31 10:00:00,10,486,0,0,1,0,59.400002,22.700001,74.0,86.199997,3.29,417.0,4001.080078,5.330591,4.7,3.928491,2.8,0.064026,69.0,1.0,100.0,0.054596,55.0,0.999996,100.0,4.160628,14.194445,12.238919,195.0,0.008889,0.0,5.841545,0.0,0.0,0.000000,0.0,0.000222,0.5,995.700012,33.0,0,5,12,0,1,0.016344,0.5,0.5,9.594916,0.0,0.0
3,1535708,1.379,2022-12-31 10:00:00,2022-12-31 10:00:00,10,486,0,0,1,0,59.400002,23.200001,74.0,86.199997,3.29,417.0,4001.080078,4.218042,3.8,2.536279,2.0,0.132874,94.0,1.0,100.0,0.371002,82.0,0.999996,100.0,1.823714,14.722222,13.567777,190.0,0.000000,0.0,5.081545,0.0,0.0,0.000000,0.0,0.000197,0.7,996.700012,33.0,0,5,12,0,1,0.016344,0.5,0.5,9.594916,0.0,0.0
4,1535708,1.379,2022-12-31 10:00:00,2022-12-31 10:00:00,10,486,0,0,1,0,59.400002,23.700001,74.0,86.199997,3.29,417.0,4001.080078,3.586084,3.0,2.155054,2.1,0.618713,95.0,1.0,100.0,1.000000,98.0,0.999996,100.0,0.730208,10.750000,12.648099,185.0,0.000000,0.0,3.925990,1.0,1.0,0.000000,0.0,0.000252,0.8,998.900024,33.0,0,5,12,0,1,0.016344,0.5,0.5,9.594916,0.0,0.0


In [35]:
train6.describe()

,row_id,target,datetime,forecast_date,hour,data_block_id,prediction_unit_id,is_business,product_type,county,latitude,longitude,lowest_price_per_mwh,highest_price_per_mwh,euros_per_mwh,eic_count,installed_capacity,ftemperature,temperature,fdewpoint,dewpoint,fcloudcover_high,cloudcover_high,fcloudcover_low,cloudcover_low,fcloudcover_mid,cloudcover_mid,fcloudcover_total,cloudcover_total,f10_metre_u_wind_component,windspeed_10m,f10_metre_v_wind_component,winddirection_10m,fdirect_solar_radiation,direct_solar_radiation,fsurface_solar_radiation_downwards,shortwave_radiation,diffuse_radiation,fsnowfall,snowfall,ftotal_precipitation,rain,surface_pressure,weather_forecast_hour,is_consumption,day_of_week,month,is_daylight,weekend,capacity_per_obs,production_obs_ratio,consumption_obs_ratio,capacity_per_capita,electricity_price_volatility_7d,gas_price_volatility_7d
count,7.821024e+06,7.821024e+06,7821024,7821024,7.821024e+06,7.821024e+06,7.821024e+06,7.821024e+06,7.821024e+06,7.821024e+06,7.821024e+06,7.821024e+06,7.821024e+06,7.821024e+06,7.821024e+06,7.821024e+06,7.821024e+06,7.821024e+06,7.821024e+06,7.821024e+06,7.821024e+06,7.821024e+06,7.821024e+06,7.821024e+06,7.821024e+06,7.821024e+06,7.821024e+06,7.821024e+06,7.821024e+06,7.821024e+06,7.821024e+06,7.821024e+06,7.821024e+06,7.821024e+06,7.821024e+06,7.821024e+06,7.821024e+06,7.821024e+06,7.821024e+06,7.821024e+06,7.821024e+06,7.821024e+06,7.821024e+06,7.821024e+06,7821024.0,7.821024e+06,7.821024e+06,7.821024e+06,7.821024e+06,7.821024e+06,7821024.0,7821024.0,7.821024e+06,7.821024e+06,7.821024e+06
mean,1.772665e+06,4.421838e+02,2023-03-15 16:23:24.042642176,2023-03-15 16:23:24.042642176,1.150334e+01,5.602036e+02,3.105827e+01,5.473324e-01,1.814040e+00,6.327553e+00,5.867327e+01,2.495431e+01,4.954704e+01,5.734236e+01,8.635880e+01,1.357397e+02,2.408381e+03,2.664004e+00,2.746641e+00,-7.600056e-01,-1.006187e+00,4.281366e-01,3.921708e+01,4.543341e-01,4.567398e+01,3.916900e-01,3.636143e+01,6.994910e-01,6.018308e+01,9.957677e-01,4.819931e+00,6.545376e-01,1.965366e+02,1.562141e+02,7.016632e+01,1.159485e+02,1.136265e+02,4.346019e+01,3.443305e-05,2.236098e-02,6.828613e-05,3.183777e-02,1.009692e+03,2.450075e+01,0.5,3.000927e+00,3.006637e+00,4.235384e-01,2.903405e-01,1.420335e-02,0.5,0.5,2.645750e+01,3.986384e+01,2.618916e+00
min,1.535708e+06,0.000000e+00,2022-12-31 10:00:00,2022-12-31 10:00:00,0.000000e+00,4.860000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.760000e+01,2.170000e+01,2.810000e+01,3.400000e+01,-1.006000e+01,5.000000e+00,5.500000e+00,-2.287803e+01,-2.170000e+01,-2.488384e+01,-2.430000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.546236e+01,0.000000e+00,-1.732074e+01,0.000000e+00,-7.111111e-01,0.000000e+00,-3.191667e-01,0.000000e+00,0.000000e+00,-3.814697e-06,0.000000e+00,-1.525879e-05,0.000000e+00,9.604000e+02,1.000000e+00,0.0,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,5.548158e-05,0.5,0.5,4.375000e-01,0.000000e+00,0.000000e+00
25%,1.653012e+06,8.790000e-01,2023-02-06 00:00:00,2023-02-06 00:00:00,6.000000e+00,5.230000e+02,1.000000e+01,0.000000e+00,1.000000e+00,2.000000e+00,5.790000e+01,2.320000e+01,4.400000e+01,4.900000e+01,4.734000e+01,1.700000e+01,4.320000e+02,-8.665527e-01,-1.100000e+00,-3.808814e+00,-4.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.624524e-01,1.900000e+01,-1.680173e+00,2.833333e+00,-2.242003e+00,1.260000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.995000e+02,1.300000e+01,0.0,1.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,2.630401e-03,0.5,0.5,1.021500e+01,3.521267e+01,1.224004e+00
50%,1.771463e+06,5.382800e+01,2023-03-15 03:00:00,2023-03-15 03:00:00,1.200000e+01,5.600000e+02,3.000000e+01,1.000000e+00,2.000000e+00,7.000000e+00,5.850000e+01,2.520000e+01,4.843000e+01,5.450000e+01,8.560000e+01,4.800000e+01,1.110500e+03,1.928980e+00,1.800000e+00,-3.448242

In [33]:
#Mae puesto n1 kaggle 52.3090
#1er primer modelo xgb sobre train5 haciendo split 80/20 de train y test 
#Mae = 107.9896
#sin lag sin feature, sin nada , solo haciendo el merge


#agrego day of week y month para todos los siguientes

#2do modelo lgb 10% y sobre esto split 80/20 de train y test 
#Mae = 89.3

#3er modelo lgb 10% y sobre esto split 80/20 de train y test 
#Mae = 103.1249
#climate_vars = ['fsurface_solar_radiation_downwards', 'shortwave_radiation', 
#                'temperature', 'direct_solar_radiation']
# agrego lags de 1h/4h/12h en las variables de arriba
# empeoro porque agregue los lags al azar no hice un analisis para agregarlo en los marcos de tiempo relevante
# estas variables de lags las quito

#agrego is_daylight binaria

#4to modelo lgb 14.2% y sobre esto split 80/20 de train y test  
#de base solo agrego la variable is_daylight , pero ahora uso info de los ultimos 3 meses 14.2% aprox
# se que no deberia ser comparable al usar de base distinto , pero parte de aca ahora
#Mae = 86.6747

#agrego weekend binaria

#5to modelo lgb igual que el anterior , solo agregue binaria Weekend y no vi cambios
#Mae = 86.6747

#para todos los demas modelos ya tienen incluido
#day of week
#month
#is_daylight
#weekend

#6to modelo lgb , agregue variable binaria is_active_hours , horario de 8 a 17 hs binaria
#Mae = 90.4776 , empeoro el modelo la saco por ahora a is_active_hours

#7mo modelo lgb, agregue variable binaria is_sleeping_hours 22h-6h
#Mae = 98.3152 empeora el modelo , asi que  la saco a is_sleeping_hours

#empiezo a probar con variables
##Energéticas:

# Ratio production/consumption por prediction_unit_id (histórico) # no se pudo probar
# Capacidad instalada per capita (installed_capacity / eic_count) # si mejora el modelo
# Eficiencia de paneles (production vs solar_radiation cuando is_consumption=False)
# Volatilidad de precios (rolling std de electricity/gas prices)
# Spread gas-electricidad (diferencia de precios, importante para switching)

#8vo modelo lgb agregue 3 variables energeticas capacity_per_obs, production_obs_ratio, consumption_obs_ratio
#Mae = 76.2703 mejor bastante el modelo

#9no modelo lgb  mantengo las 3 anteriores y prueba una nueva Ratio production/cn
# no optimizado no pudo correr

#10mo modelo lgb mantengo las 3 anteriores y pruebo  Capacidad instalada per capita (installed_capacity / eic_count)
#Mae = 72.3572 mejora el modelo 

#para los demas modelos sigo usando las siguientes variables
#capacity_per_obs
#production_obs_ratio
#consumption_obs_ratio
#capacity_per_capita

#11vo modelo lgb con esto #Eficiencia de paneles (production vs solar_radiation cuando is_consumption=False)#
#Mae anterior = 72.3572
#Mae con eficiencia de paneles = 85.0309 empeora , pruebo con log de esto  y sino no la uso por ahora


#12vo modelo lgb , lo anterior pero ahora efficiency panel es log , lo saco
# ✅ MAE con panel efficiency LOG: 85.3200
# 📊 MAE anterior: 72.3572


# #13vo modelo probando volatilidad de precio de electricidad 7 dias
# # ✅ MAE con electricity price volatility: 73.2246 si , falta probar mas versiones y gas
# # 📊 MAE anterior: 72.3572

#14vo modelo ademas de electricity_volatility pruebo con gas
# ✅ MAE con electricity + gas price volatility: 70.6517 se queda
# 📊 MAE anterior: 73.2246

#creo train6 de 150 dias para agregar
#capacity_per_obs
#production_obs_ratio
#consumption_obs_ratio
#capacity_per_capita
#elect price volatility
#gas price volatility

#15vo probar gas y/o electricity de 30 dias , agrega mucho ruido no sirve

#16vo

In [34]:
# Ratio production/consumption por prediction_unit_id (histórico) #no se pudo probar
# Capacidad instalada per capita (installed_capacity / eic_count) # si mejora el modelo
# Eficiencia de paneles (production vs solar_radiation cuando is_consumption=False) # empeora el modelo
# Volatilidad de precios (rolling std de electricity/gas prices) # mejor poco 1 cosa falta probar mas
# Volatilidad individual + combinada
# electricity_price_volatility_7d = rolling(7).std()
# gas_price_volatility_7d = rolling(7).std()
# electricity_vol_30d = rolling std de electricity prices
# gas_vol_30d = rolling std de gas prices  
# combined_price_vol = std de (electricity + gas weighted)
# price_volatility = rolling_std / rolling_mean
# Spread gas-electricidad (diferencia de precios)